<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-pipeline-components-to-use-in-SpaCy-NLP-pipeline" data-toc-modified-id="Create-pipeline-components-to-use-in-SpaCy-NLP-pipeline-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create pipeline components to use in SpaCy NLP pipeline</a></span><ul class="toc-item"><li><span><a href="#Class-to-tag-employee-nouns-as-entities" data-toc-modified-id="Class-to-tag-employee-nouns-as-entities-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Class to tag employee nouns as entities</a></span></li><li><span><a href="#YearMatcher:-Add-custom-_.is_year-attribute-to-year-tokens-that-are-part-of-Date-entities" data-toc-modified-id="YearMatcher:-Add-custom-_.is_year-attribute-to-year-tokens-that-are-part-of-Date-entities-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>YearMatcher: Add custom <code>_.is_year</code> attribute to year tokens that are part of Date entities</a></span></li></ul></li><li><span><a href="#Code-for-relationship-extraction-and-helper-functions" data-toc-modified-id="Code-for-relationship-extraction-and-helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for relationship extraction and helper functions</a></span><ul class="toc-item"><li><span><a href="#Function:-extract_emp_relations" data-toc-modified-id="Function:-extract_emp_relations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Function: extract_emp_relations</a></span></li><li><span><a href="#Testing-handling-of-tuples-(sorting-to-match-years-with-cardinal-numbers)" data-toc-modified-id="Testing-handling-of-tuples-(sorting-to-match-years-with-cardinal-numbers)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Testing handling of tuples (sorting to match years with cardinal numbers)</a></span></li><li><span><a href="#Function:-print_doc_info" data-toc-modified-id="Function:-print_doc_info-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Function: print_doc_info</a></span></li></ul></li><li><span><a href="#Define-sentence-structure-types" data-toc-modified-id="Define-sentence-structure-types-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define sentence structure types</a></span><ul class="toc-item"><li><span><a href="#Company_noun-in-subject,--Emp_num-and-EMP_NOUN-in-predicate" data-toc-modified-id="Company_noun-in-subject,--Emp_num-and-EMP_NOUN-in-predicate-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><code>Company_noun</code> in subject,  <code>Emp_num</code> and <code>EMP_NOUN</code> in predicate</a></span><ul class="toc-item"><li><span><a href="#[Company]--[was|-had]-Emp_Num-EMP_NOUN" data-toc-modified-id="[Company]--[was|-had]-Emp_Num-EMP_NOUN-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>[Company]  [<code>was</code>| <code>had</code>] <code>Emp_Num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#we-[employed-|-have]-Emp_num-EMP_NOUN" data-toc-modified-id="we-[employed-|-have]-Emp_num-EMP_NOUN-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span><code>we</code> [<code>employed</code> | <code>have</code>] <code>Emp_num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#Emp_num-below-EMP_NOUN-in-parse-tree,-org_tok-left-of-verb" data-toc-modified-id="Emp_num-below-EMP_NOUN-in-parse-tree,-org_tok-left-of-verb-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span><code>Emp_num</code> below <code>EMP_NOUN</code> in parse tree, <code>org_tok</code> left of verb</a></span></li><li><span><a href="#Org_tok-[employment_noun]-was-Emp_Num-EMP_NOUN" data-toc-modified-id="Org_tok-[employment_noun]-was-Emp_Num-EMP_NOUN-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span><code>Org_tok</code> [employment_noun] <code>was</code> <code>Emp_Num</code> <code>EMP_NOUN</code></a></span></li><li><span><a href="#Org_tok-that-is-not-the-company" data-toc-modified-id="Org_tok-that-is-not-the-company-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span><code>Org_tok</code> that is not the company</a></span></li></ul></li><li><span><a href="#Emp_noun-and-Company_noun--in-subject,-Emp_num-in-predicate" data-toc-modified-id="Emp_noun-and-Company_noun--in-subject,-Emp_num-in-predicate-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><code>Emp_noun</code> and <code>Company_noun</code>  in subject, <code>Emp_num</code> in predicate</a></span><ul class="toc-item"><li><span><a href="#&quot;The-number-of-full-time-employees-of-the-Company-was-approximately-31,800-at-December-31,-2016-and-32,300-at-December-31,-2015.&quot;" data-toc-modified-id="&quot;The-number-of-full-time-employees-of-the-Company-was-approximately-31,800-at-December-31,-2016-and-32,300-at-December-31,-2015.&quot;-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span><code>"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015."</code></a></span></li></ul></li><li><span><a href="#Emp_num-above-and-below-EMP_NOUN-in-parse-tree" data-toc-modified-id="Emp_num-above-and-below-EMP_NOUN-in-parse-tree-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span><code>Emp_num</code> above and below <code>EMP_NOUN</code> in parse tree</a></span><ul class="toc-item"><li><span><a href="#One-emp_num-is-a-conj-child-of-the-other-emp_num" data-toc-modified-id="One-emp_num-is-a-conj-child-of-the-other-emp_num-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>One emp_num is a conj child of the other emp_num</a></span></li></ul></li><li><span><a href="#Dealing-with-units" data-toc-modified-id="Dealing-with-units-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Dealing with units</a></span></li><li><span><a href="#Identifying-full-time,-part-time,-etc." data-toc-modified-id="Identifying-full-time,-part-time,-etc.-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Identifying full-time, part-time, etc.</a></span><ul class="toc-item"><li><span><a href="#As-of-February-23,-2017,-we-employed-approximately-41,000-full-time-Team-Members-and-approximately-33,000-part-time-Team-Members." data-toc-modified-id="As-of-February-23,-2017,-we-employed-approximately-41,000-full-time-Team-Members-and-approximately-33,000-part-time-Team-Members.-3.5.1"><span class="toc-item-num">3.5.1&nbsp;&nbsp;</span><code>As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.</code></a></span></li><li><span><a href="#As-of-September-30,-2016,-we-had-approximately-19,000-employees,-of-which-approximately-18,000-were-full-time-employees." data-toc-modified-id="As-of-September-30,-2016,-we-had-approximately-19,000-employees,-of-which-approximately-18,000-were-full-time-employees.-3.5.2"><span class="toc-item-num">3.5.2&nbsp;&nbsp;</span><code>As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.</code></a></span></li></ul></li><li><span><a href="#Testing-on-sentences" data-toc-modified-id="Testing-on-sentences-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Testing on sentences</a></span></li></ul></li><li><span><a href="#Testing-with-paragraphs" data-toc-modified-id="Testing-with-paragraphs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing with paragraphs</a></span></li><li><span><a href="#Matcher-testing" data-toc-modified-id="Matcher-testing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Matcher testing</a></span></li></ul></div>

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Doc, Span, Token
import pandas as pd
import re

from path import Path, getcwdu

from IPython.display import display, HTML

In [ ]:
subset_df = pd.read_excel('../data/subset_employee_count_paragraphs.xlsx')

In [ ]:
nonum_pat = re.compile(r"^([^\d]|[$][\d]+)[^\d]*$", re.I)

train_df = pd.read_csv('data/classifier_input_train.csv', index_col=0)
train_df = train_df[train_df.len > 19][train_df.para_text.apply(lambda x: not re.match(nonum_pat, x))]
val_df = pd.read_csv('data/classifier_input_val.csv', index_col=0)
val_df = val_df[val_df.len > 19][val_df.para_text.apply(lambda x: not re.match(nonum_pat, x))]

## Create pipeline components to use in SpaCy NLP pipeline

### Class to tag employee nouns as entities

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def get_case_combos(str_list):
    """Return a list with original, lower, upper, and title case."""
    case_combos = str_list + [s.lower() for s in str_list] + [s.upper() for s in str_list] + [s.title() for s in str_list]
    return list(set(case_combos))

In [ ]:
# Templated from: https://spacy.io/usage/processing-pipelines#custom-components 
class EmpNounRecognizer(object):
    """A spaCy v2.0 pipeline component that sets entity annotations
    based on list of terms. Terms are labelled as EMP_NOUN. Additionally,
    ._.has_emp_noun and ._.is_emp_noun is set on the Doc/Span and Token
    respectively."""
    name = 'employee_nouns'  # component name, will show up in the pipeline

    def __init__(self, nlp, terms=tuple(), label='EMP_NOUN'):
        """Initialise the pipeline component. The shared nlp instance is used
        to initialise the matcher with the shared vocab, get the label ID and
        generate Doc objects as phrase match patterns.
        """
        self.label = nlp.vocab.strings[label]  # get entity label ID

        # Set up the PhraseMatcher – it can now take Doc objects as patterns,
        # so even if the list of terms is long, it's very efficient
        patterns = [nlp(term) for term in terms]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add('EMP_NOUN', None, *patterns)

        # Register attribute on the Token. We'll be overwriting this based on
        # the matches, so we're only setting a default value, not a getter.
        Token.set_extension('is_emp_noun', default=False)

        # Register attributes on Doc and Span via a getter that checks if one of
        # the contained tokens is set to is_emp_noun == True.
        Doc.set_extension('has_emp_noun', getter=self.has_emp_noun)
        Span.set_extension('has_emp_noun', getter=self.has_emp_noun)

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        matches = self.matcher(doc)
        spans = []  # keep the spans for later so we can merge them afterwards
        for _, start, end in matches:
            # Generate Span representing the entity & set label
            entity = Span(doc, start, end, label=self.label)
            spans.append(entity)
            # Set custom attribute on each token of the entity
            for token in entity:
                token._.set('is_emp_noun', True)
            # Overwrite doc.ents and add entity – be careful not to replace!
            doc.ents = list(doc.ents) + [entity]
        for span in spans:
            # Iterate over all spans and merge them into one token. This is done
            # after setting the entities – otherwise, it would cause mismatched
            # indices!
            span.merge()
        return doc  # don't forget to return the Doc!

    def has_emp_noun(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is an employee noun. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_emp_noun' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_emp_noun') for t in tokens])

emp_terms_list = ["associates", "employees", "equivalents", "FTEs", "FTE's", "headcount", "individuals", 
                  "people", "persons", "team members", "workers", "workforce"]
emp_terms_list = get_case_combos(emp_terms_list)

Create entities that mark employee types (e.g., full-time, temporary, etc.)

In [8]:
# Templated from: https://spacy.io/usage/processing-pipelines#custom-components 
class EmpTypeRecognizer(object):
    """A spaCy v2.0 pipeline component that sets entity annotations
    based on list of terms. Terms are labelled as EMP_TYPE. Additionally,
    ._.has_emp_type and ._.is_emp_type is set on the Doc/Span and Token
    respectively."""
    name = 'employee_types'  # component name, will show up in the pipeline
    
    def __init__(self, nlp, terms_dict, label='EMP_TYPE'):
        """Initialise the pipeline component. The shared nlp instance is used
        to initialise the matcher with the shared vocab, get the label ID and
        generate Doc objects as phrase match patterns.
        """
        self.label = nlp.vocab.strings[label]  # get entity label ID

        # Set up the PhraseMatcher – it can now take Doc objects as patterns,
        # so even if the list of terms is long, it's very efficient
        self.matcher = PhraseMatcher(nlp.vocab)
        for match_label in terms_dict.keys():
            patterns = [nlp(term) for term in terms_dict[match_label]]
            #patterns = [nlp(term) for term in terms]
            
            self.matcher.add(match_label, None, *patterns)

        # Register attribute on the Token. We'll be overwriting this based on
        # the matches, so we're only setting a default value, not a getter.
        Token.set_extension('is_emp_type', default=False)
        Token.set_extension('is_part_time', default=False)
        Token.set_extension('is_full_time', default=False)

        # Register attributes on Doc and Span via a getter that checks if one of
        # the contained tokens is set to is_emp_type == True.
        Doc.set_extension('has_emp_type', getter=self.has_emp_type)
        Span.set_extension('has_emp_type', getter=self.has_emp_type)
        Doc.set_extension('has_part_time', getter=self.has_part_time)
        Span.set_extension('has_part_time', getter=self.has_part_time)
        Doc.set_extension('has_full_time', getter=self.has_full_time)
        Span.set_extension('has_full_time', getter=self.has_full_time)

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        matches = self.matcher(doc)
        spans = []  # keep the spans for later so we can merge them afterwards
        for match_id, start, end in matches:
            # Generate Span representing the entity & set label
            entity = Span(doc, start, end, label=match_id)
            spans.append(entity)
            # Set custom attribute on each token of the entity
            for token in entity:
                token._.set('is_emp_type', True)
                if doc.vocab.strings[match_id] == 'PART_TIME':
                    token._.set('is_part_time', True)
                elif doc.vocab.strings[match_id] == 'FULL_TIME':
                    token._.set('is_full_time', True)
                    
            # Overwrite doc.ents and add entity – be careful not to replace!
            doc.ents = list(doc.ents) + [entity]
        for span in spans:
            # Iterate over all spans and merge them into one token. This is done
            # after setting the entities – otherwise, it would cause mismatched
            # indices!
            span.merge()
        return doc  # don't forget to return the Doc!

    def has_emp_type(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is an employee type. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_emp_type' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_emp_type') for t in tokens])
    
    def has_part_time(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is indicates part time. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_part_time' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_part_time') for t in tokens])
    
    def has_full_time(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is indicates full time. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_full_time' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_full_time') for t in tokens])

In [9]:
part_time_terms = get_case_combos(["half-time", "half time", "part-time", "part time"])
full_time_terms = get_case_combos(["full-time", "full time", "40-hour equivalent", "40 hour equivalent", "full-time equivalent", "full time equivalent"])
emp_type_dict = {'PART_TIME': part_time_terms, 
                'FULL_TIME': full_time_terms}

In [10]:
#emp_phrases = [' '.join([i, j]) for i in emp_type_terms for j in emp_terms_list]
#emp_patterns = emp_phrases + emp_terms_list

In [11]:
emp_noun_recognizer = EmpNounRecognizer(nlp, emp_terms_list)
nlp.add_pipe(emp_noun_recognizer, last=True) 

emp_type_recognizer = EmpTypeRecognizer(nlp, emp_type_dict)

nlp.add_pipe(emp_type_recognizer, last=True) 

print('Pipeline', nlp.pipe_names) 

Pipeline ['tagger', 'parser', 'ner', 'employee_nouns', 'employee_types']


### YearMatcher: Add custom `_.is_year` attribute to year tokens that are part of Date entities

In [12]:
class YearMatcher(object):
    name = 'year_matcher'
    
    def __init__(self, nlp, pattern_list, match_id='Year'):
        # register a new token extension to flag year tokens
        Token.set_extension('is_year', default=False)
        self.matcher = Matcher(nlp.vocab)
        self.matcher.add(match_id, None, pattern_list)

    def __call__(self, doc):
        matches = self.matcher(doc)
        spans = []  # collect the matched spans here
        for match_id, start, end in matches:
            spans.append(doc[start:end])
        for span in spans:
            span.merge()   # merge
            for token in span:
                token._.is_year = True  # mark token as a year
        return doc

year_patterns = [{'ENT_TYPE': 'DATE', 'TAG' : 'CD', 'SHAPE' : 'dddd'}]

year_matcher = YearMatcher(nlp, year_patterns)

nlp.add_pipe(year_matcher, last=True) 

print('Pipeline', nlp.pipe_names) 

Pipeline ['tagger', 'parser', 'ner', 'employee_nouns', 'employee_types', 'year_matcher']


## Code for relationship extraction and helper functions

### Function: extract_emp_relations

In [83]:
def find_root_tok(tok):
    """Return (tok's root node, num steps to reach root)."""
    
    steps = 0 
    root_tok = tok
    while root_tok.dep_ != 'ROOT':
        steps +=1 
        root_tok = root_tok.head
    return (root_tok, steps)

def find_verb_tok(tok):
    """Return first verb ancestor of tok."""
    verb_tok = 0
    for a in tok.ancestors:
        if a.pos_ == 'VERB':
            return a
    return verb_tok
    

def find_tok_side_of_root(tok, root_tok):
    """Return 'right' or 'left' if tok is in subtree of root."""
    
    for a in [tok] + list(tok.ancestors): # The ancestors of a token will either be in root.rights or root.lefts
        if a in root_tok.lefts:
            return 'left'
        elif a in root_tok.rights:
            return 'right'
    else:
        return None

def find_subject(root_tok):
    """Return list of nominal subject"""
    subjects = [w for w in root_tok.lefts if w.dep_ == 'nsubj']
    try:
#        for i, s in enumerate(subjects):
#            print("nsubj "+str(i)+" of " + str(root_tok) + " is : " + str(s))
        return subjects[0]
    except:
        print("No nsubj found left of ROOT. Noun phrases left of root are:")
        return False
        
def get_org_span(tok):
    """Return the entity span if token has ORG ent_type_."""
    if tok.ent_type_ == 'ORG':
        subject = [e for e in tok.doc.ents if tok in e][0]
        return subject
    return tok

def check_emp_type_flags(tok):
    """Return 'Part-Time' or 'Full-Time' if corresponding flags
    are set to True."""
    
    if tok._.is_part_time == True:
        return 'Part-Time'
    if tok._.is_full_time == True:
        return 'Full-Time'
    return 'Other'

def find_emp_type_tok(tok):
    """Return child token left of tok if emp_type flagged or ADJ."""
    
    flagged_toks = [t for t in tok.children if t._.is_emp_type == True]
    if flagged_toks:
        return flagged_toks[0]
    
    candidate_tok = tok.doc[tok.i - 1]  
    while candidate_tok.is_punct == True:
        candidate_tok = tok.doc[candidate_tok.i - 1]
    if candidate_tok.head == tok:
        if candidate_tok.pos_ == 'ADJ' or candidate_tok.dep_ == 'compound':
            return candidate_tok
    return 0

def get_nummod_tok(tok, years, verbose=True):
    """Return tok.children that are nummod and card entities."""
    
    num_toks = [c for c in tok.children if c.dep_ == 'nummod' and c.ent_type_ == 'CARDINAL']
    if num_toks:
        if verbose == True:
            print("Num_toks are: " + str(num_toks))
        num_tok = num_toks[0]
        num_tok_conj = [c for c in num_tok.children if c.dep_=='conj' and c.tag_ == 'CD']
        
        if num_tok_conj:
            if verbose == True:
                print("num_tok has conjugate children:" + str(num_tok_conj))
                print("num_tok subtree is :" + str(list(num_tok.subtree)))
            cards = [(c.i, c) for c in num_tok.subtree if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
            
            if len(years) == len(cards):
                order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                year_emps = [(years[i][1].text, cards[i][1].text) for i in order_indices]
                num_tok = max(year_emps)[1]
    #            print("years: " + str(years))
     #           print("cards: " + str(cards))
      #          print("order_indices: " + str(order_indices))
                if verbose == True:
                    print("year_emps: " + str(year_emps))     
        return num_tok
    
    return 0      

def extract_emp_relations(doc, verbose=True):
    
    relations = []
    
    for sent_id, sent in enumerate(doc.sents):
        
        # Find the root token
        root_tok, depth = find_root_tok(sent[0])

        # If verb does not have expected lemma, move to next sentence
        if root_tok.lemma_ not in ['be', 'employ', 'have']:
            if verbose == True:
                print("Root token lemma not one of ['be', 'employ', 'have']. ")
                print("Root token, lemma are : " + str(root_tok) + " " + str(root_tok.lemma_))
                print(list(root_tok.subtree))
            continue

        match_pairs = []
        relation_tuples = []
        num_tok, num_tok_conj, year_conj = (False, False, False)
        years = [(y.i, y) for y in root_tok.subtree if y._.is_year == True] # Need to change to root.subtree to only return the word's sentence

        for word_id, word in enumerate(filter(lambda w: w.ent_type_ == 'EMP_NOUN', sent)):  

            verb_tok = find_verb_tok(word)
            emp_type_tok = find_emp_type_tok(word)
            emp_type = 'Other'
            if emp_type_tok:
                emp_type = check_emp_type_flags(emp_type_tok)
            parts_found = []
            # Find out if the employee noun is in subject (left) or predicate (right)
            left_side = []; right_side = []
            emp_tok_side = find_tok_side_of_root(word, root_tok)
            if emp_tok_side == 'left':
                left_side.append(word)
            elif emp_tok_side == 'right':
                right_side.append(word)
            else:
                if verbose == True:
                    print("No ancestor of'" + str(word) + "' is in root.rights or root.lefts.")    

            if verbose == True:
                print("Dep_ of EMP_NOUN is: " + str(word.dep_))
            if word.dep_ in ('attr', 'dobj') or word.dep_ == 'pobj' and word.head.dep_ == 'prep':
                num_tok = get_nummod_tok(word, years, verbose = verbose)      
                if num_tok:
                    match_pairs.append((num_tok, word))
                else:
                    cards = [e for e in word.doc.ents if e.label_ == 'CARDINAL' and e.root in root_tok.rights]
                    if cards:
                        cards = cards + [c for c in word.doc.ents if c.root in cards[0].root.subtree and c not in cards and c.label_ == 'CARDINAL']           
                        if word in left_side:    
                            if len(years) > 0:                       
                                emp_counts = [(c.start, c) for c in cards]                      
                                order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                                try: 
                                    year_emps = [(years[i][1].text, emp_counts[i][1].text) for i in order_indices]
                                    num_tok = max(year_emps)[1]
                                    if verbose == True:
                                        print("year_emps: " + str(year_emps))
                                        print("max year_emps: " + str(max(year_emps)))
                                except:
                                    print("Length of emp_counts is : " + str(len(emp_counts)) + 
                                         " while length of years is : " + str(len(years)))
                                    num_tok = cards[0]
                                if verbose == True:
                                    print("Sentence has multiple years:" + str(years))
                                    print("First card subtree is :" + str(list(cards[0].subtree)))
                                    print("years: " + str(years))
                                    print("cards: " + str(cards))
                                    print("emp_counts: " + str(emp_counts))
                    #                print("order_indices: " + str(order_indices))                   
                                match_pairs.append((num_tok, word))
                        else:
                            if verbose == True:
                                print("Emp_tok is in right side; appending first card.")
                            match_pairs.append((cards[0], word))
                    elif verb_tok.dep_ == 'relcl':
                        cards = [e for e in word.doc.ents if e.label_ == 'CARDINAL' and e.root in verb_tok.lefts]
                        if cards:
                            match_pairs.append((cards[0], word))
                            num_tok = cards[0]
                if verbose == True:
                    print("Root is at "+str(depth)+" steps from "+str(word)+".")
                subject = find_subject(root_tok)
                if not subject: # For debugging
                    if verbose == True:
                        print("No nsubj found left of ROOT. Noun phrases left of root are:")
                    left_filter = lambda e: e.root in root_tok.lefts
                    print_df(make_span_df(doc, entities=False, span_filter_func=left_filter))
                else:
                    subject = get_org_span(subject) # Use full span of ORG entity if subject tok is in ORG                    
                    parts_found.append(subject)
                    match_pairs.append((subject, word))
                    #[print(str(p) + '  :  ' + str(p.dep_)) for p in subject.subtree]
                    sub_poss = [p for p in subject.subtree if p.dep_ == 'poss']
    #                if sub_poss:
    #                    sub_poss = sub_poss[0]
    #                    match_pairs.append((sub_poss, word))
                    if root_tok:
                        parts_found.append(root_tok)
                    if num_tok:
                        parts_found.append(num_tok)
                        parts_found.append(emp_type)
                        parts_found.append(emp_type_tok)
                        parts_found.append(word)
                    elif word.head.head.head.pos_ == 'VERB':
                        print("No num_tok. ")
                        cards = [c for c in word.head.head.head.rights if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
                        years = [(y.i, y) for y in root_tok.subtree if y._.is_year == True]
                        match_pairs.append((years, cards))
                        if cards:
                            match_pairs.append((cards[0], word))

            elif word.dep_ == 'conj':
                num_tok = get_nummod_tok(word, years, verbose = verbose)
                head_num_tok = [w for w in [word.head] if w.tag_ == 'CD']
                if verbose == True:
                    print("Emp_noun token has dep_ == 'conj'.")
                    print("Child num_tok: " + str(child_num_tok))
                    print("Head num_tok: " + str(head_num_tok))
                if num_tok and head_num_tok:
                    if verbose == True:
                        print("child_num_tok and head_num_toks")
                    years = [(y.i, y) for y in root_tok.subtree if y._.is_year == True]
                    emp_counts = [(c.i, c) for c in [child_num_tok] + head_num_tok]
                    order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                    year_emps = [(years[i][1], emp_counts[i][1]) for i in order_indices]
                    num_tok = max(year_emps)[1]
                match_pairs.append((num_tok, word))
                
                if not subject:
                    subject = get_org_span(find_subject(root_tok))
                parts_found.append(subject)
                parts_found.append(root_tok)
                parts_found.append(num_tok)
                parts_found.append(emp_type)
                parts_found.append(emp_type_tok)
                parts_found.append(word)
            
            if all([subject, root_tok, num_tok, emp_type ]):
                relation_tuples.append(tuple([sent_id, word_id] + parts_found))
            if verbose == True:
                print(tuple(parts_found))
        
        relations.append(relation_tuples)
    return relations


Initial work on defining a class, although I just want functions right now

In [303]:
class EmployeeRelationsExtractor(object):
    """A class to manage different sentence structures
    and multiple relations within a sentence."""

    def find_root_tok(self, doc):
        """Return (tok's root node, num steps to reach root)."""  
        steps = 0 
        root_tok = doc[0]
        while root_tok.dep_ != 'ROOT':
            steps +=1 
            root_tok = root_tok.head
        return (root_tok, steps)    

    def __init__(self, doc):
        self.root_tok, self.depth = self.find_root_tok(doc)
        self.years = [(y.i, y) for y in self.root_tok.subtree if y._.is_year == True] # Need to change to root.subtree to only return the word's sentence
        self.org_ents = [e for e in doc.ents if e.label_ == 'ORG']

    def __call__(self, doc):
        print(self.root_tok)
        root_tok = self.root_tok
        
        num_tok, num_tok_conj, year_conj = (False, False, False)
        dates = [e for e in root_tok.doc.ents if e.label_ == 'DATE']
        relations = []
        parts_found = []
        if root_tok.lemma_ not in ['be', 'employ', 'have']:
            print("Root token lemma not one in ['be', 'employ', 'have']. ")
            print("Root token, lemma are : " + str(self.root_tok) + " " + str(self.root_tok.lemma_))
            return list(self.root_tok.subtree)
        
        for word in filter(lambda w: w.ent_type_ == 'EMP_NOUN', doc):  
      
        # Find out if the employee noun is in subject (left) or predicate (right)
            left_side = []; right_side = []
            emp_tok_side = find_tok_side_of_root(word, root_tok)
            if emp_tok_side == 'left':
                left_side.append(word)
            elif emp_tok_side == 'right':
                right_side.append(word)
            else:
                print("No ancestor of'" + str(word) + "' is in root.rights or root.lefts.")    

In [305]:

EmployeeRelationsExtractor(ex1).__call__()

had


NameError: name 'doc' is not defined

In [134]:
def extract_emp_relations_old(doc):
    relations = []
    parts_found = []
    org_ents = [e for e in doc.ents if e.label_ == 'ORG']
    num_tok, num_tok_conj, year_conj = (False, False, False)
    for word in filter(lambda w: w.ent_type_ == 'EMP_NOUN', doc):
        
        # Find the root token
        depth=1
        root_tok = word.head
        while root_tok.dep_ != 'ROOT':
                    root_tok = root_tok.head
                    depth +=1
        
        # Find out if the employee noun is in subject (left) or predicate (right)
        left_side = []; right_side = []
        if depth == 1:
            if word in root_tok.lefts:
                left_side.append(word)
            elif word in root_tok.rights:
                left_side.append(word)
            else:
                print(str(word) + " is in root.rights or root.lefts.")
        else:
            for w in word.ancestors:
                if w in root_tok.lefts:
                    print("Ancestor '" + str(w) + "' of emp_tok is in root.lefts")
                    left_side.append(word)
                    break
                elif w in root_tok.rights:
                    print("Ancestor '" + str(w) + "' of emp_tok is in root.rights")
                    right_side.append(word)
                    break
            else:
                print("No ancestor of'" + str(word) + "' is in root.rights or root.lefts.")
                    
        if word.dep_ in ('attr', 'dobj'):
            possible_root = word.head
            num_toks = [w for w in word.children if w.dep_ == 'nummod']
        elif word.dep_ == 'pobj' and word.head.dep_ == 'prep':
            possible_root = word.head.head 
            depth=2
            num_toks = [c for c in word.children if c.dep_ == 'nummod' and c.ent_type_ == 'CARDINAL']
#        if org_ents:
#                relations.append((org_ents[0], word))
#        if word.dep_ in ('attr', 'dobj'):
        if possible_root:
            if num_toks:
                print("Num_toks are: " + str(num_toks))
                num_tok = num_toks[0]
                num_tok_conj = [c for c in num_tok.children if c.dep_=='conj' and c.tag_ == 'CD']
                if len(num_tok_conj) > 0:
                    print("num_tok has conjugate children:" + str(num_tok_conj))
                    print("num_tok subtree is :" + str(list(num_tok.subtree)))
                    cards = [c for c in num_tok.subtree if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
                    emp_counts = [(c.i, c) for c in cards]
                    years = [(y.i, y) for y in word.doc if y._.is_year == True] # Need to change to root.subtree to only return the word's sentence
                    order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                    year_emps = [(years[i][1], emp_counts[i][1]) for i in order_indices]
                    relations.append((max(year_emps)[1], word))
                else:
#                    dates = [e for e in word.doc.ents if e.label_ == 'DATE']
                    relations.append((num_tok, word))
            elif word.head.head.head.pos_ == 'VERB':
                cards = [c for c in word.head.head.head.rights if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
                dates = [e for e in word.doc.ents if e.label_ == 'DATE']
                if cards:
                    relations.append((cards[0], word))
            # If EMP_NOUN head is a verb and root, find the nsubj
            if possible_root.dep_ == 'ROOT':
                root_tok = possible_root
                subjects = [w for w in root_tok.lefts if w.dep_ == 'nsubj']
                for i, s in enumerate(subjects):
                    print("nsubj "+str(i)+" of " + str(root_tok) + " is : " + str(s))
                if not subjects:
                    print("No nsubj found left of ROOT. Noun phrases left of root are:")
                    left_filter = lambda e: e.root in root_tok.lefts
                    print_df(make_span_df(doc, entities=False, span_filter_func=left_filter))
            else:
                print(str(word)+" with "+str(depth)+" climbs is not root. ")
                emp_tok_filter = lambda t: t == word
                print_df(make_tok_df(doc, tok_filter_func=emp_tok_filter))
                root_tok = possible_root
                while root_tok.dep_ != 'ROOT':
                    root_tok = root_tok.head
                    depth +=1
                print("Root is at "+str(depth)+" climbs from "+str(word)+".")
                subjects = [w for w in root_tok.lefts if w.dep_ == 'nsubj']
                for i, s in enumerate(subjects):
                    print("nsubj "+str(i)+" of " + str(root_tok) + " is : " + str(s))
            if subjects:
                subject = subjects[0]
                if subject.ent_type_ == 'ORG':
                    # Use full span of ORG entity if subject tok is in ORG
                    subject = [e for e in doc.ents if subject in e][0]
                parts_found.append(subject)
                relations.append((subject, word))
                #[print(str(p) + '  :  ' + str(p.dep_)) for p in subject.subtree]
                sub_poss = [p for p in subject.subtree if p.dep_ == 'poss']
#                if sub_poss:
#                    sub_poss = sub_poss[0]
#                    relations.append((sub_poss, word))
                if root_tok:
                    parts_found.append(root_tok)
                if num_tok:
                    num_tok_conj = [c for c in num_tok.children if c.dep_=='conj' and c.tag_ == 'CD']
                    if len(num_tok_conj) > 0:
                        print("num_tok has conjugate children:" + str(num_tok_conj[0]))
                    parts_found.append(num_tok)
                    parts_found.append(word)
                elif word.head.head.head.pos_ == 'VERB':
                    cards = [c for c in word.head.head.head.rights if c.tag_== 'CD' and c.ent_type_ == 'CARDINAL']
                    dates = [e for e in word.doc.ents if e.label_ == 'DATE']
                    years = [(y.i, y) for y in word.doc if y._.is_year == True]
                    relations.append((years, cards))
                    if cards:
                        relations.append((cards[0], word))
                
            
#        elif word.dep_ == 'pobj' and word.head.dep_ == 'prep':
#            subject = word.head.head 
#            relations.append((subject, word))
        elif word.dep_ == 'conj':
            child_num_toks = [w for w in word.children if w.dep_ == 'nummod'] 
            head_num_tok = [w for w in [word.head] if w.tag_ == 'CD']
            if child_num_toks and head_num_tok:
                years = [(y.i, y) for y in word.doc if y._.is_year == True]
                emp_counts = [(c.i, c) for c in child_num_toks + head_num_tok]
                order_indices = [years.index(y) for y in sorted(years, reverse=True, key = lambda x: x[1])]
                year_emps = [(years[i][1], emp_counts[i][1]) for i in order_indices]
                relations.append((max(year_emps)[1], word))
            else: 
                relations.append((child_num_toks[0], word))
        relations.append(tuple(parts_found))
    return relations


### Testing handling of tuples (sorting to match years with cardinal numbers)

In [10]:
iyears =[(13, 2015), (11, 2016)]
iemps = [(15, '43,246'), (12, '53,146')]
year_sorted = sorted(iyears, key = lambda tup: tup[1], reverse=True)
order_indices = [iyears.index(x) for x in year_sorted]
year_emps = [(iyears[i][1], iemps[i][1]) for i in order_indices]

sorted(iemps)
year_sorted
order_indices
year_emps
max(year_emps)[1]

'53,146'

### Function: print_doc_info 

In [14]:
def make_tok_df(doc, tok_filter_func=False):
    """Return a dataframe showing attributes for each token in doc."""
    
    if tok_filter_func:
        toks = list(filter(tok_filter_func, doc))
    else:
        toks = doc
    doc_dict = {'tok_ent' : [tok.ent_type_ for tok in toks], 
        'toks' : [tok for tok in toks], 
        'lemma' : [tok.lemma_ for tok in toks], 
        'dep' : [tok.dep_ for tok in toks], 
        'head' : [tok.head for tok in toks], 
        'h_dep' : [tok.head.dep_ for tok in toks], 
        'dep_def' : [spacy.explain(tok.dep_) for tok in toks],
        'pos' : [tok.pos_ for tok in toks], 
        'tag' : [tok.tag_ for tok in toks], 
        'tag_def' : [spacy.explain(tok.tag_) for tok in toks], 
               }
    columns = [ 'tok_ent', 'toks', 'lemma', 'dep', 'head', 'h_dep', 'pos', 'tag',  'dep_def', 'tag_def' ]
    return pd.DataFrame(doc_dict, columns=columns)


def make_span_df(doc, entities=True, span_filter_func=False):
    """Return df showing attributes for each entity or noun chunk in doc."""
    
    columns = ['tok_i', 'entity', 'ent_label', 'root', 'root_ent', 
               'root_dep','dep_def', 'root_head',  'root_head_dep', 
               'root_head_pos'  ]
    if entities:
        target_spans = doc.ents
        df_name = 'doc_entities'
    else: 
        target_spans = list(doc.noun_chunks)
        df_name = 'doc_noun_chunks'
        
    if span_filter_func:
        target_spans = list(filter(span_filter_func, target_spans))
        
    doc_dict = {'tok_i' : [e.start for e in target_spans], 
        'entity' : [e.text for e in target_spans], 
        'ent_label' : [e.label_ for e in target_spans], 
        'root' : [e.root.text for e in target_spans], 
        'root_ent' : [e.root.ent_type_ for e in target_spans], 
        'root_dep' : [e.root.dep_ for e in target_spans], 
        'dep_def' : [spacy.explain(e.root.dep_) for e in target_spans],
        'root_head' : [e.root.head for e in target_spans], 
        'root_head_dep' : [e.root.head.dep_ for e in target_spans], 
        'root_head_pos' : [e.root.head.pos_ for e in target_spans]}
#    try :
    df = pd.DataFrame(doc_dict, columns=columns) 
    df.name = df_name
    if entities:
        df_cols = [x for x in df.columns.tolist() if x != 'root_ent']
        df = df[df_cols]
    else:
        df.columns = ['tok_i', 'noun_chunk', 'ent_label', 'root', 'root_ent', 'root_dep','dep_def',
                      'root_head',  'root_head_dep', 'root_head_pos' ]
        df_cols = [x for x in df.columns.tolist() if x != 'ent_label']
        df = df[df_cols]
#    except: 
#        df = [(k, doc_dict[k]) for k in doc_dict.keys()]
    return df

def print_df(df, print_df_name=True):
    """Display rendered HTML version of df in Jupyter notebook"""
    
    if print_df_name and hasattr(df, 'name'):
        print("DataFrame is named " + str(df.name))
    display(HTML(df.to_html()))


def print_doc_info(doc):
    """Print dfs for doc entities, noun_chunks, and CARDINAL toks"""
    
    print("doc is: ")
    print(doc)
    print('-' * 50)
    print("Entities are: ")
    print_df(make_span_df(doc), print_df_name=False)
    print('-' * 50)
    print("Noun chunks are: ")
    print_df(make_span_df(doc, entities=False), print_df_name=False)
    print('-' * 50)
    print("Cardinal entities are: ")
    card_filter = lambda w: w.ent_type_ == 'CARDINAL'
    print_df(make_tok_df(doc, card_filter), print_df_name=False)

## Define sentence structure types

### `Company_noun` in subject,  `Emp_num` and `EMP_NOUN` in predicate

#### [Company]  [`was`| `had`] `Emp_Num` `EMP_NOUN`

Examples: 


`"At December 31, 2016, Bio-Rad had approximately 8,250 employees."`

Desired output: 

`(Bio-Rad, had, 8,250, employees)`

In [82]:
ex1 = nlp('At December 31, 2016, Bio-Rad had approximately 8,250 employees.')

In [83]:
displacy.render(ex1, style='ent', jupyter=True, options={'distance': 90})

In [84]:
displacy.render(ex1, style='dep', jupyter=True, options={'distance': 90})

In [85]:
print_df(make_tok_df(ex1))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,At,at,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,DATE,December,december,pobj,At,prep,PROPN,NNP,object of preposition,"noun, proper singular"
2,DATE,31,31,nummod,December,pobj,NUM,CD,None,cardinal number
3,DATE,",",",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
4,DATE,2016,2016,nummod,December,pobj,NUM,CD,None,cardinal number
5,,",",",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
6,ORG,Bio,bio,compound,Rad,nsubj,PROPN,NNP,None,"noun, proper singular"
7,ORG,-,-,punct,Rad,nsubj,PUNCT,HYPH,punctuation,"punctuation mark, hyphen"
8,ORG,Rad,rad,nsubj,had,ROOT,PROPN,NNP,nominal subject,"noun, proper singular"
9,,had,have,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"


In [88]:
print_doc_info(ex1)

doc is: 
At December 31, 2016, Bio-Rad had approximately 8,250 employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,"December 31, 2016",DATE,December,pobj,object of preposition,At,prep,ADP
1,6,Bio-Rad,ORG,Rad,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 8,250",CARDINAL,"8,250",nummod,None,employees,dobj,NOUN
3,12,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,December,December,DATE,pobj,object of preposition,At,prep,ADP
1,6,Bio-Rad,Rad,ORG,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 8,250 employees",employees,EMP_NOUN,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,"8,250",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"8,250","8,250",nummod,employees,dobj,NUM,CD,None,cardinal number


In [89]:
ex1_emp_tok = ex1[12]

In [90]:
ex1_emp_tok.dep_

'dobj'

In [91]:
[x for x in ex1_emp_tok.children]

[8,250]

In [101]:
list(list(ex1_emp_tok.head.lefts)[0].subtree)

[At, December, 31, ,, 2016]

In [104]:
#[w for w in ex1_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex1_emp_tok) )
for c in ex1_emp_tok.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex1_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex1_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex1_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex1_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex1_emp_tok.head.i))
    print('Token head is: ' + str(ex1_emp_tok.head) + '       head.dep_:  '+ str(ex1_emp_tok.head.dep_) )
    for i, d in enumerate(ex1_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex1_emp_tok.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex1_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex1_emp_tok.dep_))
print('Token head: ' + str(ex1_emp_tok.head))
print('       head.dep_: ' + str(ex1_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex1_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex1_emp_tok.head.pos_) + "'" )

print('Token conjuncts: ' )
print(str([c for c in ex1_emp_tok.conjuncts]))
print('Token head lefts: ' )
for w in ex1_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token head rights: ' )
for w in ex1_emp_tok.head.rights:
    print(str(w) + '       rights.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex1_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex1_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 8,250  has dep_ == 'nummod'
       Token.child.i : 11
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: dobj
Token head: had
       head.dep_: ROOT
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token conjuncts: 
[]
Token head lefts: 
At       head.left.dep_:prep
,       head.left.dep_:punct
Rad       head.left.dep_:nsubj
Rad is nsubj. Subtree is:
Bio
-
Token head rights: 
employees       rights.left.dep_:dobj
.       rights.left.dep_:punct
Token ancestors: 
had       ancestor.dep_:ROOT
Token children: 
8,250       child.dep_:nummod


In [235]:
extract_emp_relations(ex1)

Dep_ of EMP_NOUN is: employees
Num_toks are: [8,250]
nsubj 0 of had is : Rad
Root is at 1 steps from employees.


[(8,250, employees), (Bio-Rad, employees), (Bio-Rad, had, 8,250, employees)]

#### `we` [`employed` | `have`] `Emp_num` `EMP_NOUN`

`"As of December 31, 2016, the Company had 455 employees, an increase of 17 employees from the prior year end."`

Desired output: 

`(Company, had, 455, employees)`

In [192]:
ex2 = nlp("As of December 31, 2016, the Company had 455 employees, an increase of 17 employees from the prior year end.")

In [193]:
print_df(make_tok_df(ex2))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,As,as,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,December,december,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,31,31,nummod,December,pobj,NUM,CD,None,cardinal number
4,DATE,",",",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2016,2016,nummod,December,pobj,NUM,CD,None,cardinal number
6,,",",",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,the,the,det,Company,nsubj,DET,DT,determiner,determiner
8,,Company,company,nsubj,had,ROOT,PROPN,NNP,nominal subject,"noun, proper singular"
9,,had,have,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"


In [195]:
print_doc_info(ex2)

doc is: 
As of December 31, 2016, the Company had 455 employees, an increase of 17 employees from the prior year end.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"December 31, 2016",DATE,December,pobj,object of preposition,of,prep,ADP
1,10,455,CARDINAL,455,nummod,None,employees,dobj,NOUN
2,11,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB
3,16,17,CARDINAL,17,nummod,None,employees,pobj,NOUN
4,17,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP
5,19,the prior year end,DATE,end,pobj,object of preposition,from,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,December,December,DATE,pobj,object of preposition,of,prep,ADP
1,7,the Company,Company,,nsubj,nominal subject,had,ROOT,VERB
2,10,455 employees,employees,EMP_NOUN,dobj,direct object,had,ROOT,VERB
3,13,an increase,increase,,appos,appositional modifier,employees,dobj,NOUN
4,16,17 employees,employees,EMP_NOUN,pobj,object of preposition,of,prep,ADP
5,19,the prior year end,end,DATE,pobj,object of preposition,from,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,455,455,nummod,employees,dobj,NUM,CD,None,cardinal number
1,CARDINAL,17,17,nummod,employees,pobj,NUM,CD,None,cardinal number


In [194]:
ex2_emp_tok = ex2[11]

In [200]:
ex2_emp_tok_2 = ex2[17]
ex2_emp_tok_2

employees

In [197]:
displacy.render(ex2, style='dep', jupyter=True)

In [196]:
# Properties of the first emp_tok
print('Token:  ' + str(ex2_emp_tok) )
for c in ex2_emp_tok.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex2_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex2_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex2_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex2_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex2_emp_tok.head.i))
    print('Token head is: ' + str(ex2_emp_tok.head) + '       head.dep_:  '+ str(ex2_emp_tok.head.dep_) )
    for i, d in enumerate(ex2_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex2_emp_tok.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex2_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex2_emp_tok.dep_))
print('Token head: ' + str(ex2_emp_tok.head))
print('       head.dep_: ' + str(ex2_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex2_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex2_emp_tok.head.pos_) + "'" )

print('Token conjuncts: ' )
print(str([c for c in ex2_emp_tok.conjuncts]))
print('Token head lefts: ' )
for w in ex2_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token head rights: ' )
for w in ex2_emp_tok.head.rights:
    print(str(w) + '       rights.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex2_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex2_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 455  has dep_ == 'nummod'
       Token.child.i : 10
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: dobj
Token head: had
       head.dep_: ROOT
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token conjuncts: 
[]
Token head lefts: 
As       head.left.dep_:prep
,       head.left.dep_:punct
Company       head.left.dep_:nsubj
Company is nsubj. Subtree is:
the
Token head rights: 
employees       rights.left.dep_:dobj
.       rights.left.dep_:punct
Token ancestors: 
had       ancestor.dep_:ROOT
Token children: 
455       child.dep_:nummod
,       child.dep_:punct
increase       child.dep_:appos


In [201]:
# Properties of the second emp_tok
print('Token:  ' + str(ex2_emp_tok_2) )
for c in ex2_emp_tok_2.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex2_emp_tok_2.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex2_emp_tok_2.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex2_emp_tok_2.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex2_emp_tok_2.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex2_emp_tok_2.head.i))
    print('Token head is: ' + str(ex2_emp_tok_2.head) + '       head.dep_:  '+ str(ex2_emp_tok_2.head.dep_) )
    for i, d in enumerate(ex2_emp_tok_2.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex2_emp_tok_2.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex2_emp_tok_2.head.dep_ == ''

print('Token dep_: ' + str(ex2_emp_tok_2.dep_))
print('Token head: ' + str(ex2_emp_tok_2.head))
print('       head.dep_: ' + str(ex2_emp_tok_2.head.dep_))
print("       head.tag_: '"+ str(ex2_emp_tok_2.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex2_emp_tok_2.head.pos_) + "'" )

print('Token conjuncts: ' )
print(str([c for c in ex2_emp_tok_2.conjuncts]))
print('Token head lefts: ' )
for w in ex2_emp_tok_2.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token head rights: ' )
for w in ex2_emp_tok_2.head.rights:
    print(str(w) + '       rights.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex2_emp_tok_2.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex2_emp_tok_2.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 17  has dep_ == 'nummod'
       Token.child.i : 16
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: pobj
Token head: of
       head.dep_: prep
       head.tag_: 'IN'
       head.pos_: 'ADP'
Token conjuncts: 
[]
Token head lefts: 
Token head rights: 
employees       rights.left.dep_:pobj
Token ancestors: 
of       ancestor.dep_:prep
increase       ancestor.dep_:appos
employees       ancestor.dep_:dobj
had       ancestor.dep_:ROOT
Token children: 
17       child.dep_:nummod


In [198]:
extract_emp_relations(ex2)

Num_toks are: [455]
nsubj 0 of had is : Company
Root is at 1 steps from employees.
Ancestor 'employees' of emp_tok is in root.rights
Num_toks are: [17]
nsubj 0 of had is : Company
Root is at 4 steps from employees.


[(455, employees),
 (Company, employees),
 (Company, had, 455, employees),
 (17, employees),
 (Company, employees),
 (Company, had, 455, employees, Company, had, 17, employees)]

#### `Emp_num` below `EMP_NOUN` in parse tree, `org_tok` left of verb

Example sentece:

`"As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees."`

Desired output: 

`(AEP, had, 17,634, employees)`

In [45]:
ex3 = nlp("As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.")

In [46]:
[tok for tok in ex3 if tok.ent_type_ == 'ORG']

[AEP]

In [47]:
print_df(make_tok_df(ex3))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,As,as,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,December,december,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,31,31,nummod,December,pobj,NUM,CD,None,cardinal number
4,DATE,",",",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2016,2016,nummod,December,pobj,NUM,CD,None,cardinal number
6,,",",",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,the,the,det,subsidiaries,nsubj,DET,DT,determiner,determiner
8,,subsidiaries,subsidiary,nsubj,had,ROOT,NOUN,NNS,nominal subject,"noun, plural"
9,,of,of,prep,subsidiaries,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"


In [48]:
print_doc_info(ex3)

doc is: 
As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"December 31, 2016",DATE,December,pobj,object of preposition,of,prep,ADP
1,10,AEP,ORG,AEP,pobj,object of preposition,of,prep,ADP
2,15,"17,634",CARDINAL,"17,634",nummod,None,employees,pobj,NOUN
3,16,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,December,December,DATE,pobj,object of preposition,of,prep,ADP
1,7,the subsidiaries,subsidiaries,,nsubj,nominal subject,had,ROOT,VERB
2,10,AEP,AEP,ORG,pobj,object of preposition,of,prep,ADP
3,12,a total,total,,dobj,direct object,had,ROOT,VERB
4,15,"17,634 employees",employees,EMP_NOUN,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,"17,634","17,634",nummod,employees,pobj,NUM,CD,None,cardinal number


In [130]:
ex3_emp_tok = ex3[16]

ex3_emp_num_ent = ex3.ents[2]

In [133]:
ex3_emp_num_ent.root.is_digit

False

In [51]:
ex3_emp_num_ent.root.ent_type_

'CARDINAL'

In [52]:
ex3[3].ent_type_

'DATE'

In [120]:
#[w for w in ex3_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex3_emp_tok) )
for c in ex3_emp_tok.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex3_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex3_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex3_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex3_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex3_emp_tok.head.i))
    print('Token head is: ' + str(ex3_emp_tok.head) + '       head.dep_:  '+ str(ex3_emp_tok.head.dep_) )
    for i, d in enumerate(ex3_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex3_emp_tok.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex3_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex3_emp_tok.dep_))
print('Token head: ' + str(ex3_emp_tok.head))
print('Token head: ' + str(ex3_emp_tok.head.head))
print('       head.dep_: ' + str(ex3_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex3_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex3_emp_tok.head.pos_) + "'" )

print('Token conjuncts: ' )
print(str([c for c in ex3_emp_tok.conjuncts]))
print('Token root lefts: ' )
for w in ex3_emp_tok.head.head.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token head rights: ' )
for w in ex3_emp_tok.head.head.head.rights:
    print(str(w) + '       rights.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex3_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Check for ancestor in root.rights: ')
for w in ex3_emp_tok.ancestors:
    if w in ex3_emp_tok.head.head.head.lefts:
        print("Ancestor '" + str(w) + "' of emp_tok is in root.lefts")
        break
    elif w in ex3_emp_tok.head.head.head.lefts:
        print("Ancestor '" + str(w) + "' of emp_tok is in root.rights")
        break
    else:
        print("Ancestor '" + str(w) + "' of emp_tok is not in tok.rights or tok.lefts.")
else:
    print("No ancestor of emp_tok is not in tok.rights or tok.lefts.")
print('Token children: ')
for w in ex3_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 17,634  has dep_ == 'nummod'
       Token.child.i : 15
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: pobj
Token head: of
Token head: total
       head.dep_: prep
       head.tag_: 'IN'
       head.pos_: 'ADP'
Token conjuncts: 
[]
Token root lefts: 
As       head.left.dep_:prep
,       head.left.dep_:punct
subsidiaries       head.left.dep_:nsubj
subsidiaries is nsubj. Subtree is:
the
Token head rights: 
total       rights.left.dep_:dobj
.       rights.left.dep_:punct
Token ancestors: 
of       ancestor.dep_:prep
total       ancestor.dep_:dobj
had       ancestor.dep_:ROOT
Check for ancestor in root.rights: 
Ancestor 'of' of emp_tok is not in tok.rights or tok.lefts.
Ancestor 'total' of emp_tok is not in tok.rights or tok.lefts.
Ancestor 'had' of emp_tok is not in tok.rights or tok.lefts.
No ancestor of emp_tok is not in tok.rights or tok.lefts.
Token children: 
17,634       child.dep_:nummod


In [54]:
displacy.render(ex3, style='dep', jupyter=True, options={'distance': 110})

In [55]:
print(ex3_emp_tok.head)
print(ex3_emp_tok.head.dep_)
print(ex3_emp_tok.head.head)
print(ex3_emp_tok.head.head.dep_)
print(ex3_emp_tok.head.head.head)
print(ex3_emp_tok.head.head.head.dep_)
print(ex3_emp_tok.head.head.head.pos_)
print(ex3_emp_tok.head.head.head.tag_)
print([c for c in ex3_emp_tok.head.head.head.children])
print([c.dep_ for c in ex3_emp_tok.head.head.head.children])
print([c.tag_ for c in ex3_emp_tok.head.head.head.children])
print([r for r in ex3_emp_tok.head.head.head.subtree if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex3_emp_tok.head.head.head.children])
print([e for e in ex3.ents])
print([e.label_ for e in ex3.ents])
print([e for e in ex3.ents if e.label_ == 'DATE'])

of
prep
total
dobj
had
ROOT
VERB
VBD
[As, ,, subsidiaries, total, .]
['prep', 'punct', 'nsubj', 'dobj', 'punct']
['IN', ',', 'NNS', 'NN', '.']
[31, 2016, 17,634]
['', '', '', '', '']
[December 31, 2016, AEP, 17,634, employees]
['DATE', 'ORG', 'CARDINAL', 'EMP_NOUN']
[December 31, 2016]


In [56]:
list(ex3[8].subtree)

[the, subsidiaries, of, AEP]

In [138]:
extract_emp_relations(ex3)

Ancestor 'total' of emp_tok is in root.rights
Num_toks are: [17,634]
nsubj 0 of had is : subsidiaries
Root is at 3 climbs from employees.


[(17,634, employees),
 (subsidiaries, employees),
 (subsidiaries, had, 17,634, employees)]

#### `Org_tok` [employment_noun] `was` `Emp_Num` `EMP_NOUN`

`"Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries."`

Desired output: 

`(employment, was, 14,000, employees)`

In [53]:
ex4 = nlp("Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.")

In [54]:
print_df(make_tok_df(ex4))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,ORG,Alcoa,poss,employment,nsubj,PROPN,NNP,possession modifier,"noun, proper singular"
1,,'s,case,Alcoa,poss,PART,POS,None,possessive ending
2,,total,amod,employment,nsubj,ADJ,JJ,adjectival modifier,adjective
3,,worldwide,amod,employment,nsubj,ADJ,JJ,adjectival modifier,adjective
4,,employment,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
5,,at,prep,employment,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
6,DATE,the,det,end,pobj,DET,DT,determiner,determiner
7,DATE,end,pobj,at,prep,NOUN,NN,object of preposition,"noun, singular or mass"
8,DATE,of,prep,end,pobj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
9,DATE,2016,pobj,of,prep,NUM,CD,object of preposition,cardinal number


In [55]:
print_doc_info(ex4)

doc is: 
Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,Alcoa,ORG,Alcoa,poss,possession modifier,employment,nsubj,NOUN
1,6,the end of 2016,DATE,end,pobj,object of preposition,at,prep,ADP
2,11,"approximately 14,000",CARDINAL,"14,000",nummod,None,employees,attr,NOUN
3,13,employees,EMP_NOUN,employees,attr,attribute,was,ROOT,VERB
4,15,15,CARDINAL,15,nummod,None,countries,pobj,NOUN


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,Alcoa's total worldwide employment,NP,employment,nsubj,nominal subject,was,ROOT,VERB
1,6,the end,NP,end,pobj,object of preposition,at,prep,ADP
2,11,"approximately 14,000 employees",NP,employees,attr,attribute,was,ROOT,VERB
3,15,15 countries,NP,countries,pobj,object of preposition,in,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,advmod,"14,000",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"14,000",nummod,employees,attr,NUM,CD,None,cardinal number
2,CARDINAL,15,nummod,countries,pobj,NUM,CD,None,cardinal number


In [56]:
ex4_emp_tok = ex4[13]

In [57]:
#[w for w in ex4_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token dep_: ' + str(ex4_emp_tok.dep_))
print('Token head: ' + str(ex4_emp_tok.head) + '       head.dep_:'+ str(ex4_emp_tok.head.dep_) )
for w in ex4_emp_tok.head.lefts:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex4_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex4_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token dep_: attr
Token head: was       head.dep_:ROOT
employment       head.left.dep_:nsubj
employment is nsubj. Subtree is:
Alcoa
total
worldwide
Alcoa
Token ancestors: 
was       ancestor.dep_:ROOT
Token children: 
14,000       child.dep_:nummod
in       child.dep_:prep


In [58]:
displacy.render(ex4, style='dep', jupyter=True, options={'distance': 90})

In [153]:
extract_emp_relations(ex4)

nsubj 0 of was is : employment


[(14,000, employees),
 (employment, employees),
 (employment, was, 14,000, employees)]

#### `Org_tok` that is not the company

Desired output: 

`(Company, employed, 1,562, persons)`

In [60]:
ex5 = nlp("The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.")

In [61]:
print_df(make_tok_df(ex5))

,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,,The,det,Company,nsubj,DET,DT,determiner,determiner
1,,Company,nsubj,employed,ROOT,PROPN,NNP,nominal subject,"noun, proper singular"
2,,and,cc,Company,nsubj,CCONJ,CC,coordinating conjunction,"conjunction, coordinating"
3,,its,poss,subsidiaries,conj,ADJ,PRP$,possession modifier,"pronoun, possessive"
4,,subsidiaries,conj,Company,nsubj,NOUN,NNS,conjunct,"noun, plural"
5,,employed,ROOT,employed,ROOT,VERB,VBD,None,"verb, past tense"
6,CARDINAL,"1,562",nummod,persons,dobj,NUM,CD,None,cardinal number
7,EMP_NOUN,persons,dobj,employed,ROOT,NOUN,NNS,direct object,"noun, plural"
8,,at,prep,employed,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
9,DATE,December,pobj,at,prep,PROPN,NNP,object of preposition,"noun, proper singular"


In [62]:
print_doc_info(ex5)

doc is: 
The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,6,"1,562",CARDINAL,"1,562",nummod,None,persons,dobj,NOUN
1,7,persons,EMP_NOUN,persons,dobj,direct object,employed,ROOT,VERB
2,9,"December 31, 2016",DATE,December,pobj,object of preposition,at,prep,ADP
3,14,114,CARDINAL,114,nsubjpass,nominal subject (passive),covered,relcl,VERB
4,25,District 10,FAC,District,pobj,object of preposition,with,prep,ADP
5,28,the International Association of Machinists,ORG,Association,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,NP,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The Company,NP,Company,nsubj,nominal subject,employed,ROOT,VERB
1,3,its subsidiaries,NP,subsidiaries,conj,conjunct,Company,nsubj,PROPN
2,6,"1,562 persons",NP,persons,dobj,direct object,employed,ROOT,VERB
3,9,December,NP,December,pobj,object of preposition,at,prep,ADP
4,16,whom,NP,whom,pobj,object of preposition,of,prep,ADP
5,20,a collective bargaining agreement,NP,agreement,pobj,object of preposition,by,agent,ADP
6,25,District,NP,District,pobj,object of preposition,with,prep,ADP
7,28,the International Association,NP,Association,pobj,object of preposition,of,prep,ADP
8,32,Machinists,NP,Machinists,pobj,object of preposition,of,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,dep,head,head_dep,pos,tag,dep_def,tag_def
0,CARDINAL,"1,562",nummod,persons,dobj,NUM,CD,None,cardinal number
1,CARDINAL,114,nsubjpass,covered,relcl,NUM,CD,nominal subject (passive),cardinal number


In [63]:
ex5_emp_tok = ex5[7]

In [64]:
ex5_emp_tok.dep_

'dobj'

In [65]:
displacy.render(ex5, style='dep', jupyter=True, options={'distance': 110})

In [66]:
print(ex5_emp_tok.head)
print(ex5_emp_tok.head.dep_)
print([c for c in ex5_emp_tok.head.lefts])
print([c.dep_ for c in ex5_emp_tok.head.head.head.children])
print([c.tag_ for c in ex5_emp_tok.head.head.head.children])
print([r for r in ex5_emp_tok.head.head.head.rights if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex5_emp_tok.head.head.head.children])
print([e for e in ex5.ents])
print([e.label_ for e in ex5.ents])
print([e for e in ex5.ents if e.label_ == 'DATE'])

employed
ROOT
[Company]
['nsubj', 'dobj', 'prep', 'punct']
['NNP', 'NNS', 'IN', '.']
[]
['', 'B', '', '']
[1,562, persons, December 31, 2016, 114, District 10, the International Association of Machinists]
['CARDINAL', 'EMP_NOUN', 'DATE', 'CARDINAL', 'FAC', 'ORG']
[December 31, 2016]


In [154]:
extract_emp_relations(ex5)

nsubj 0 of employed is : Company


[(1,562, persons), (Company, persons), (Company, employed, 1,562, persons)]

### `Emp_noun` and `Company_noun`  in subject, `Emp_num` in predicate

#### `"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015."`

Examples:  
`"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015."`

Desired output:  
`(number, was, 31,800, full-time employees)`

In [12]:
ex6 = nlp("The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.")

In [13]:
print_df(make_tok_df(ex6))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,The,the,det,number,nsubj,DET,DT,determiner,determiner
1,,number,number,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
2,,of,of,prep,number,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
3,FULL_TIME,full-time,full,compound,employees,pobj,ADJ,JJ,None,adjective
4,EMP_NOUN,employees,employee,pobj,of,prep,NOUN,NNS,object of preposition,"noun, plural"
5,,of,of,prep,employees,pobj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
6,,the,the,det,Company,pobj,DET,DT,determiner,determiner
7,ORG,Company,company,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
8,,was,be,ROOT,was,ROOT,VERB,VBD,None,"verb, past tense"
9,CARDINAL,approximately,approximately,advmod,"31,800",attr,ADV,RB,adverbial modifier,adverb


In [14]:
print_doc_info(ex6)

doc is: 
The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,3,full-time,FULL_TIME,full-time,compound,None,employees,pobj,NOUN
1,4,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP
2,7,Company,ORG,Company,pobj,object of preposition,of,prep,ADP
3,9,"approximately 31,800",CARDINAL,"31,800",attr,attribute,was,ROOT,VERB
4,12,"December 31, 2016",DATE,December,pobj,object of preposition,at,prep,ADP
5,17,"32,300",CARDINAL,"32,300",conj,conjunct,was,ROOT,VERB
6,19,"December 31, 2015",DATE,December,pobj,object of preposition,at,prep,ADP


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The number,number,,nsubj,nominal subject,was,ROOT,VERB
1,3,full-time employees,employees,EMP_NOUN,pobj,object of preposition,of,prep,ADP
2,6,the Company,Company,ORG,pobj,object of preposition,of,prep,ADP
3,12,December,December,DATE,pobj,object of preposition,at,prep,ADP
4,19,December,December,DATE,pobj,object of preposition,at,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,"31,800",attr,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"31,800","31,800",attr,was,ROOT,NUM,CD,attribute,cardinal number
2,CARDINAL,"32,300","32,300",conj,was,ROOT,NUM,CD,conjunct,cardinal number


In [15]:
ex6_emp_tok = ex6[4]

In [16]:
ex6_emp_tok.dep_

'pobj'

In [17]:
print([c for c in ex6[4].children])

[full-time, of]


In [18]:
print('Token children: ')
for w in ex6[10].children:
    print(str(w) + '       child.dep_:' + str(w.dep_))
    
print('Token children: ')
for w in ex6[17].children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token children: 
approximately       child.dep_:advmod
Token children: 
at       child.dep_:prep


In [19]:
[c for c in ex6[8].conjuncts]

[32,300]

In [20]:
print(ex6_emp_tok.head.dep_)
print(ex6_emp_tok.head.head)
print(ex6_emp_tok.head.head.dep_)
print(ex6_emp_tok.head.head.head)
print(ex6_emp_tok.head.head.head.dep_)
print(ex6_emp_tok.head.head.head.pos_)
print(ex6_emp_tok.head.head.head.tag_)
print([c for c in ex6_emp_tok.head.head.head.children])
print([c.dep_ for c in ex6_emp_tok.head.head.head.children])
print([c.tag_ for c in ex6_emp_tok.head.head.head.children])
print([r for r in ex6_emp_tok.head.head.head.rights if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex6_emp_tok.head.head.head.children])
print([e for e in ex6.ents])
print([e.label_ for e in ex6.ents])
print([e for e in ex6.ents if e.label_ == 'DATE'])

prep
number
nsubj
was
ROOT
VERB
VBD
[number, 31,800, at, and, 32,300, .]
['nsubj', 'attr', 'prep', 'cc', 'conj', 'punct']
['NN', 'CD', 'IN', 'CC', 'CD', '.']
[31,800, 32,300]
['', 'I', '', '', 'B', '']
[full-time, employees, Company, approximately 31,800, December 31, 2016, 32,300, December 31, 2015]
['FULL_TIME', 'EMP_NOUN', 'ORG', 'CARDINAL', 'DATE', 'CARDINAL', 'DATE']
[December 31, 2016, December 31, 2015]


In [21]:
displacy.render(ex6, style='dep', jupyter=True, options={'distance': 110})

In [22]:
extract_emp_relations(ex6)

NameError: name 'extract_emp_relations' is not defined

Example 5

### `Emp_num` above and below `EMP_NOUN` in parse tree

#### One emp_num is a conj child of the other emp_num

Example text:

`At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.`

Desired output: 

`(we, had, 56,400, employees)`

In [141]:
ex7 = nlp("At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.")

In [142]:
print_df(make_tok_df(ex7))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,At,at,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,DATE,December,december,pobj,At,prep,PROPN,NNP,object of preposition,"noun, proper singular"
2,DATE,31,31,nummod,December,pobj,NUM,CD,None,cardinal number
3,DATE,",",",",punct,December,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
4,DATE,2016,2016,nummod,December,pobj,NUM,CD,None,cardinal number
5,DATE,and,and,cc,2016,nummod,CCONJ,CC,coordinating conjunction,"conjunction, coordinating"
6,DATE,2015,2015,conj,2016,nummod,NUM,CD,conjunct,cardinal number
7,,",",",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
8,,we,-PRON-,nsubj,had,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
9,,had,have,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"


In [143]:
print_doc_info(ex7)

doc is: 
At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,"December 31, 2016 and 2015",DATE,December,pobj,object of preposition,At,prep,ADP
1,10,"approximately 56,400",CARDINAL,"56,400",nummod,None,employees,dobj,NOUN
2,13,"66,400",CARDINAL,"66,400",conj,conjunct,"56,400",nummod,NUM
3,14,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,1,December,December,DATE,pobj,object of preposition,At,prep,ADP
1,8,we,we,,nsubj,nominal subject,had,ROOT,VERB
2,10,"approximately 56,400 and 66,400 employees",employees,EMP_NOUN,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,"56,400",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"56,400","56,400",nummod,employees,dobj,NUM,CD,None,cardinal number
2,CARDINAL,"66,400","66,400",conj,"56,400",nummod,NUM,CD,conjunct,cardinal number


In [144]:
ex7_emp_tok = ex7[14]

ex7_emp_num_tok = ex7[11]

In [145]:
ex7_emp_tok

employees

In [146]:
ex7_emp_tok.dep_

'dobj'

In [147]:
ex7_emp_num_tok

56,400

In [148]:
[c for c in ex7_emp_num_tok.conjuncts]

[66,400]

In [149]:
print('Token children: ')
for w in ex7_emp_num_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token children: 
approximately       child.dep_:advmod
and       child.dep_:cc
66,400       child.dep_:conj


In [150]:
#[w for w in ex7_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex7_emp_tok) )
for c in ex7_emp_tok.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex7_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex7_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex7_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex7_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex7_emp_tok.head.i))
    print('Token head is: ' + str(ex7_emp_tok.head) + '       head.dep_:  '+ str(ex7_emp_tok.head.dep_) )
    for i, d in enumerate(ex7_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex7_emp_tok.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex7_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex7_emp_tok.dep_))
print('Token head: ' + str(ex7_emp_tok.head))
print('       head.dep_: ' + str(ex7_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex7_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex7_emp_tok.head.pos_) + "'" )
print('Token head lefts: ' )
print('Token conjuncts: ' )
print(str([c for c in ex7_emp_tok.conjuncts]))
for w in ex7_emp_tok.head.rights:
    print(str(w) + '       head.left.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex7_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex7_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token child: 56,400  has dep_ == 'nummod'
       Token.child.i : 11
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
       Token GC dep_ is conj.
       Token GC is 66,400
       gc.tag_ == 'CD' 
       gc.i : 13
       Token head is: 56,400       head.dep_:  nummod
0  December 31, 2016 and 2015
       CD tok in date: 2016
       CD tok in date: 2015
Token dep_: dobj
Token head: had
       head.dep_: ROOT
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token head lefts: 
Token conjuncts: 
[]
employees       head.left.dep_:dobj
,       head.left.dep_:punct
respectively       head.left.dep_:advmod
.       head.left.dep_:punct
Token ancestors: 
had       ancestor.dep_:ROOT
Token children: 
56,400       child.dep_:nummod


In [151]:
displacy.render(ex7, style='dep', jupyter=True, options={'distance': 110})

In [152]:
print(ex7_emp_tok.head)
print(ex7_emp_tok.head.dep_)
print(ex7_emp_tok.head.head)
print(ex7_emp_tok.head.head.dep_)
print(ex7_emp_tok.head.head.head)
print(ex7_emp_tok.head.head.head.dep_)
print(ex7_emp_tok.head.head.head.pos_)
print(ex7_emp_tok.head.head.head.tag_)
print([c for c in ex7_emp_tok.head.head.head.children])
print([c.dep_ for c in ex7_emp_tok.head.head.head.children])
print([c.tag_ for c in ex7_emp_tok.head.head.head.children])
print([r for r in ex7_emp_tok.head.head.head.subtree if r.tag_== 'CD'])
print([c.ent_iob_ for c in ex7_emp_tok.head.head.head.children])
print([e for e in ex7.ents])
print([e.label_ for e in ex7.ents])
print([e for e in ex7.ents if e.label_ == 'DATE'])

had
ROOT
had
ROOT
had
ROOT
VERB
VBD
[At, ,, we, employees, ,, respectively, .]
['prep', 'punct', 'nsubj', 'dobj', 'punct', 'advmod', 'punct']
['IN', ',', 'PRP', 'NNS', ',', 'RB', '.']
[31, 2016, 2015, 56,400, 66,400]
['', '', '', 'B', '', '', '']
[December 31, 2016 and 2015, approximately 56,400, 66,400, employees]
['DATE', 'CARDINAL', 'CARDINAL', 'EMP_NOUN']
[December 31, 2016 and 2015]


In [160]:
max([(2015, 66,400), (2016, 56,400)])

(2016, 56, 400)

In [169]:
extract_emp_relations(ex7)

Num_toks are: [56,400]
num_tok has conjugate children:[66,400]
num_tok subtree is :[approximately, 56,400, and, 66,400]
year_emps: [('2015', '66,400'), ('2016', '56,400')]
nsubj 0 of had is : we
Root is at 1 climbs from employees.


[('56,400', employees), (we, employees), (we, had, '56,400', employees)]

### Dealing with units

Example text:  

`"The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively."`

In [78]:
thousands_sent_doc = nlp("The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively.")

In [79]:
print_df(make_tok_df(thousands_sent_doc))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,The,the,det,number,nsubj,DET,DT,determiner,determiner
1,,number,number,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
2,,of,of,prep,number,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
3,,regular,regular,amod,employees,pobj,ADJ,JJ,adjectival modifier,adjective
4,EMP_NOUN,employees,employee,pobj,of,prep,NOUN,NNS,object of preposition,"noun, plural"
5,,was,be,ROOT,was,ROOT,VERB,VBD,None,"verb, past tense"
6,CARDINAL,71.1,71.1,compound,thousand,attr,NUM,CD,None,cardinal number
7,CARDINAL,thousand,thousand,attr,was,ROOT,NUM,CD,attribute,cardinal number
8,,",",",",punct,thousand,attr,PUNCT,",",punctuation,"punctuation mark, comma"
9,CARDINAL,73.5,73.5,compound,thousand,appos,NUM,CD,None,cardinal number


In [14]:
list(list(thousands_sent_doc[4].head.head.head.rights)[0].subtree)

[71.1, thousand, ,, 73.5, thousand, ,, and, 75.3, thousand]

In [15]:
[r for r in thousands_sent_doc[4].head.head.head.lefts]

[number]

In [80]:
print_doc_info(thousands_sent_doc)

doc is: 
The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,4,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP
1,6,71.1 thousand,CARDINAL,thousand,attr,attribute,was,ROOT,VERB
2,9,73.5 thousand,CARDINAL,thousand,appos,appositional modifier,thousand,attr,NUM
3,13,75.3 thousand,CARDINAL,thousand,conj,conjunct,thousand,attr,NUM
4,16,years ended 2016,DATE,ended,advcl,adverbial clause modifier,was,ROOT,VERB
5,20,2015,DATE,2015,conj,conjunct,2016,npadvmod,NUM
6,22,2014,DATE,2014,conj,conjunct,2015,conj,NUM


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The number,number,,nsubj,nominal subject,was,ROOT,VERB
1,3,regular employees,employees,EMP_NOUN,pobj,object of preposition,of,prep,ADP
2,16,years,years,DATE,pobj,object of preposition,at,prep,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,71.1,71.1,compound,thousand,attr,NUM,CD,None,cardinal number
1,CARDINAL,thousand,thousand,attr,was,ROOT,NUM,CD,attribute,cardinal number
2,CARDINAL,73.5,73.5,compound,thousand,appos,NUM,CD,None,cardinal number
3,CARDINAL,thousand,thousand,appos,thousand,attr,NUM,CD,appositional modifier,cardinal number
4,CARDINAL,75.3,75.3,compound,thousand,conj,NUM,CD,None,cardinal number
5,CARDINAL,thousand,thousand,conj,thousand,attr,NUM,CD,conjunct,cardinal number


In [17]:
displacy.render(thousands_sent_doc, style='ent', jupyter=True)

In [18]:
print_df(make_tok_df(nlp("We had approximately 2.1 thousand employees.")))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,We,-PRON-,nsubj,had,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
1,,had,have,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"
2,CARDINAL,approximately,approximately,advmod,thousand,nummod,ADV,RB,adverbial modifier,adverb
3,CARDINAL,2.1,2.1,compound,thousand,nummod,NUM,CD,None,cardinal number
4,CARDINAL,thousand,thousand,nummod,employees,dobj,NUM,CD,None,cardinal number
5,EMP_NOUN,employees,employee,dobj,had,ROOT,NOUN,NNS,direct object,"noun, plural"
6,,.,.,punct,had,ROOT,PUNCT,.,punctuation,"punctuation mark, sentence closer"


In [19]:
print_doc_info(nlp("We had approximately 2.1 thousand employees."))

doc is: 
We had approximately 2.1 thousand employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,approximately 2.1 thousand,CARDINAL,thousand,nummod,None,employees,dobj,NOUN
1,5,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,We,We,,nsubj,nominal subject,had,ROOT,VERB
1,2,approximately 2.1 thousand employees,employees,EMP_NOUN,dobj,direct object,had,ROOT,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,thousand,nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,2.1,2.1,compound,thousand,nummod,NUM,CD,None,cardinal number
2,CARDINAL,thousand,thousand,nummod,employees,dobj,NUM,CD,None,cardinal number


In [20]:
displacy.render(nlp("we had 2.1 thousand employees."), jupyter=True)

In [21]:
displacy.render(thousands_sent_doc, jupyter=True)

In [82]:
extract_emp_relations(thousands_sent_doc)

Dep_ of EMP_NOUN is: pobj
year_emps: [('2014', '75.3 thousand'), ('2015', '73.5 thousand'), ('2016', '71.1 thousand')]
max year_emps: ('2016', '71.1 thousand')
Sentence has multiple years:[(18, 2016), (20, 2015), (22, 2014)]
First card subtree is :[71.1, thousand, ,, 73.5, thousand, ,, and, 75.3, thousand]
years: [(18, 2016), (20, 2015), (22, 2014)]
cards: [71.1 thousand, 73.5 thousand, 75.3 thousand]
emp_counts: [(6, 71.1 thousand), (9, 73.5 thousand), (13, 75.3 thousand)]
Root is at 2 steps from employees.
(number, was, '71.1 thousand', 'Other', regular, employees)


[[(0, 0, number, was, '71.1 thousand', 'Other', regular, employees)]]

### Identifying full-time, part-time, etc.

#### ```As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.```

Example text:

`As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.`  

Desired output: 

`(we, employed, 41,000, full-time Team Members)`   
`(we, employed, 33,000, part-time Team Members)`

In [23]:
ex8 = nlp("As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.")

In [24]:
print_df(make_tok_df(ex8))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,As,as,prep,employed,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,February,february,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,23,23,nummod,February,pobj,NUM,CD,None,cardinal number
4,DATE,",",",",punct,February,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2017,2017,nummod,February,pobj,NUM,CD,None,cardinal number
6,,",",",",punct,employed,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,we,-PRON-,nsubj,employed,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
8,,employed,employ,ROOT,employed,ROOT,VERB,VBD,None,"verb, past tense"
9,CARDINAL,approximately,approximately,advmod,"41,000",nummod,ADV,RB,adverbial modifier,adverb


In [63]:
print_doc_info(ex8)

doc is: 
As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"February 23, 2017",DATE,February,pobj,object of preposition,of,prep,ADP
1,9,"approximately 41,000",CARDINAL,"41,000",nummod,None,Team Members,dobj,PROPN
2,11,full-time,FULL_TIME,full-time,compound,None,Team Members,dobj,PROPN
3,12,Team Members,EMP_NOUN,Team Members,dobj,direct object,employed,ROOT,VERB
4,14,"approximately 33,000",CARDINAL,"33,000",nummod,None,Team Members,conj,PROPN
5,16,part-time,PART_TIME,part-time,compound,None,Team Members,conj,PROPN
6,17,Team Members,EMP_NOUN,Team Members,conj,conjunct,Team Members,dobj,PROPN


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,February,February,DATE,pobj,object of preposition,of,prep,ADP
1,7,we,we,,nsubj,nominal subject,employed,ROOT,VERB
2,9,"approximately 41,000 full-time Team Members",Team Members,EMP_NOUN,dobj,direct object,employed,ROOT,VERB
3,14,"approximately 33,000 part-time Team Members",Team Members,EMP_NOUN,conj,conjunct,Team Members,dobj,PROPN


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,"41,000",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"41,000","41,000",nummod,Team Members,dobj,NUM,CD,None,cardinal number
2,CARDINAL,approximately,approximately,advmod,"33,000",nummod,ADV,RB,adverbial modifier,adverb
3,CARDINAL,"33,000","33,000",nummod,Team Members,conj,NUM,CD,None,cardinal number


In [26]:
ex8_emp_tok = ex8[12]
ex8_emp_num_tok = ex8[10]
ex8_emp_tok_2 = ex8[17]

In [53]:
ex8[ex8_emp_tok.i - 1]

full-time

In [70]:
check_emp_type_flags(find_emp_type_tok(ex8_emp_tok))

'Full-Time'

In [48]:
[t for t in ex8_emp_tok.children if t._.is_emp_type == True]

'compound'

In [49]:
[t for t in ex8_emp_tok_2.children if t._.is_emp_type == True][0].pos_

'ADJ'

In [18]:
ex8_emp_tok.dep_

'dobj'

In [28]:
ex8_emp_num_tok

41,000

In [29]:
[c for c in ex8_emp_tok.conjuncts]

[Team Members]

In [30]:
print('Token children: ')
for w in ex8_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token children: 
41,000       child.dep_:nummod
full-time       child.dep_:compound
and       child.dep_:cc
Team Members       child.dep_:conj


In [31]:
#[w for w in ex8_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex8_emp_tok) )
for c in ex8_emp_tok.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex8_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex8_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex8_emp_tok.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex8_emp_tok.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex8_emp_tok.head.i))
    print('Token head is: ' + str(ex8_emp_tok.head) + '       head.dep_:  '+ str(ex8_emp_tok.head.dep_) )
    for i, d in enumerate(ex8_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex8_emp_tok.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex8_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex8_emp_tok.dep_))
print('Token head: ' + str(ex8_emp_tok.head))
print('       head.dep_: ' + str(ex8_emp_tok.head.dep_))
print("       head.tag_: '"+ str(ex8_emp_tok.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex8_emp_tok.head.pos_) + "'" )
print('Token head lefts: ' )
print('Token conjuncts: ' )
print(str([c for c in ex8_emp_tok.conjuncts]))
for w in ex8_emp_tok.head.rights:
    print(str(w) + '       head.right.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex8_emp_tok.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex8_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  Team Members
Token child: 41,000  has dep_ == 'nummod'
       Token.child.i : 10
       child.dep_: nummod
       child.tag_: 'CD'
       child.pos_: 'NUM'
Token dep_: dobj
Token head: employed
       head.dep_: ROOT
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token head lefts: 
Token conjuncts: 
[Team Members]
Team Members       head.right.dep_:dobj
.       head.right.dep_:punct
Token ancestors: 
employed       ancestor.dep_:ROOT
Token children: 
41,000       child.dep_:nummod
full-time       child.dep_:compound
and       child.dep_:cc
Team Members       child.dep_:conj


In [32]:
displacy.render(ex8, style='dep', jupyter=True, options={'distance': 110})

In [33]:
displacy.render(ex8, style='ent', jupyter=True, options={'distance': 110})

In [72]:
extract_emp_relations(ex8)

Dep_ of EMP_NOUN is: dobj
Num_toks are: [41,000]
Root is at 1 steps from Team Members.
Dep_ of EMP_NOUN is: conj
Emp_noun token has dep_ == 'conj'.
Num_toks are: [33,000]
Child num_tok: 33,000
Head num_tok: []


[(we, employed, 41,000, (full-time, 'Full-Time'), Team Members),
 (we, employed, 33,000, (part-time, 'Part-Time'), Team Members)]

#### ```As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.```

Example text:

`As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.`  

Desired output: 

`(we, had, 19,000, (0, 'Other') ,employees)`  
`(we, had, 18,000, ('full-time', 'Full-Time') ,employees)`  

In [78]:
ex9 = nlp("As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.")

In [80]:
print_df(make_tok_df(ex9))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,As,as,prep,had,ROOT,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
1,,of,of,prep,As,prep,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
2,DATE,September,september,pobj,of,prep,PROPN,NNP,object of preposition,"noun, proper singular"
3,DATE,30,30,nummod,September,pobj,NUM,CD,None,cardinal number
4,DATE,",",",",punct,September,pobj,PUNCT,",",punctuation,"punctuation mark, comma"
5,DATE,2016,2016,nummod,September,pobj,NUM,CD,None,cardinal number
6,,",",",",punct,had,ROOT,PUNCT,",",punctuation,"punctuation mark, comma"
7,,we,-PRON-,nsubj,had,ROOT,PRON,PRP,nominal subject,"pronoun, personal"
8,,had,have,ROOT,had,ROOT,VERB,VBD,None,"verb, past tense"
9,CARDINAL,approximately,approximately,advmod,"19,000",nummod,ADV,RB,adverbial modifier,adverb


In [81]:
print_doc_info(ex9)

doc is: 
As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,"September 30, 2016",DATE,September,pobj,object of preposition,of,prep,ADP
1,9,"approximately 19,000",CARDINAL,"19,000",nummod,None,employees,dobj,NOUN
2,11,employees,EMP_NOUN,employees,dobj,direct object,had,ROOT,VERB
3,15,"approximately 18,000",CARDINAL,"18,000",nsubj,nominal subject,were,relcl,VERB
4,18,full-time,FULL_TIME,full-time,compound,None,employees,attr,NOUN
5,19,employees,EMP_NOUN,employees,attr,attribute,were,relcl,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,2,September,September,DATE,pobj,object of preposition,of,prep,ADP
1,7,we,we,,nsubj,nominal subject,had,ROOT,VERB
2,9,"approximately 19,000 employees",employees,EMP_NOUN,dobj,direct object,had,ROOT,VERB
3,18,full-time employees,employees,EMP_NOUN,attr,attribute,were,relcl,VERB


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,approximately,approximately,advmod,"19,000",nummod,ADV,RB,adverbial modifier,adverb
1,CARDINAL,"19,000","19,000",nummod,employees,dobj,NUM,CD,None,cardinal number
2,CARDINAL,approximately,approximately,advmod,"18,000",nsubj,ADV,RB,adverbial modifier,adverb
3,CARDINAL,"18,000","18,000",nsubj,were,relcl,NUM,CD,nominal subject,cardinal number


In [87]:
ex9_emp_tok = ex9[11]
ex9_emp_num_tok = ex9[16]
ex9_emp_tok_2 = ex9[19]

In [88]:
ex9[ex9_emp_tok.i - 1]

19,000

In [89]:
check_emp_type_flags(find_emp_type_tok(ex9_emp_tok_2))

'Full-Time'

In [94]:
[t for t in ex9_emp_tok.children if t._.is_emp_type == True]

[]

In [93]:
[t for t in ex9_emp_tok_2.children if t._.is_emp_type == True]

[full-time]

In [18]:
ex9_emp_tok.dep_

'dobj'

In [28]:
ex9_emp_num_tok

41,000

In [95]:
[c for c in ex9_emp_tok.conjuncts]

[]

In [96]:
print('Token children: ')
for w in ex9_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token children: 
19,000       child.dep_:nummod
,       child.dep_:punct
were       child.dep_:relcl


In [109]:
#[w for w in ex9_emp_tok.head.lefts if w.dep_ == 'nsubj']
print('Token:  ' + str(ex9_emp_tok_2) )
for c in ex9_emp_tok_2.children:
    if c.dep_ == 'nummod':
        print("Token child: " + str(c) + "  has dep_ == 'nummod'")
        print("       Token.child.i : " + str(c.i))
        print('       child.dep_: ' + str(c.dep_))
        print("       child.tag_: '"+ str(c.tag_) + "'" )
        print("       child.pos_: '"+ str(c.pos_) + "'" )
        for gc in c.children:
            if gc.dep_ == 'conj':
                print('       Token GC dep_ is conj.')
                print('       Token GC is ' + str(gc))
                if gc.tag_ == 'CD':
                    print("       gc.tag_ == 'CD' ")
                    print("       gc.i : " + str(gc.i))
                print('       Token head is: ' + str(c) + '       head.dep_:  '+ str(c.dep_) )
                for i, d in enumerate(ex9_emp_tok.doc.ents):
                    if d.label_ == 'DATE':
                        print(str(i) + "  " + str(d))
                        for tok in ex9_emp_tok.doc[d.start:d.end + 1]:
                            if tok._.is_year == True:
                                print("       CD tok in date: " + str(tok))
                
if ex9_emp_tok_2.dep_ == 'conj':
    print('Token dep_ is conj.')
    if ex9_emp_tok_2.head.tag_ == 'CD':
        print("       token.head.tag_ == 'CD' ")
        print("       token.head.i : " + str(ex9_emp_tok.head.i))
    print('Token head is: ' + str(ex9_emp_tok.head) + '       head.dep_:  '+ str(ex9_emp_tok.head.dep_) )
    for i, d in enumerate(ex9_emp_tok.doc.ents):
        if d.label_ == 'DATE':
            print(str(i) + "  " + str(d))
            for tok in ex9_emp_tok_2.doc[d.start:d.end + 1]:
                if tok._.is_year == True:
                    print("CD tok in date: " + str(tok))
                    print("CD tok i: " + str(tok.i))
#    if ex9_emp_tok.head.dep_ == ''

print('Token dep_: ' + str(ex9_emp_tok_2.dep_))
print('Token head: ' + str(ex9_emp_tok_2.head))
print('       head.dep_: ' + str(ex9_emp_tok_2.head.dep_))
print("       head.tag_: '"+ str(ex9_emp_tok_2.head.tag_) + "'" )
print("       head.pos_: '"+ str(ex9_emp_tok_2.head.pos_) + "'" )
print('Token head lefts: ' )
print(str([c for c in ex9_emp_tok_2.head.lefts]))
print('Token conjuncts: ' )
print(str([c for c in ex9_emp_tok.conjuncts]))
for w in ex9_emp_tok.head.rights:
    print(str(w) + '       head.right.dep_:' + str(w.dep_))
    if w.dep_ == 'nsubj':
        print(str(w) + ' is nsubj. Subtree is:' )
        [print(x) for x in w.lefts]
        [print(s)for s in w.subtree if s.dep_ == 'poss']
print('Token ancestors: ')
for w in ex9_emp_tok_2.ancestors:
    print(str(w) + '       ancestor.dep_:' + str(w.dep_))
print('Token children: ')
for w in ex9_emp_tok.children:
    print(str(w) + '       child.dep_:' + str(w.dep_))

Token:  employees
Token dep_: attr
Token head: were
       head.dep_: relcl
       head.tag_: 'VBD'
       head.pos_: 'VERB'
Token head lefts: 
[18,000]
Token conjuncts: 
[]
employees       head.right.dep_:dobj
.       head.right.dep_:punct
Token ancestors: 
were       ancestor.dep_:relcl
employees       ancestor.dep_:dobj
had       ancestor.dep_:ROOT
Token children: 
19,000       child.dep_:nummod
,       child.dep_:punct
were       child.dep_:relcl


In [98]:
displacy.render(ex9, style='dep', jupyter=True, options={'distance': 110})

In [33]:
displacy.render(ex9, style='ent', jupyter=True, options={'distance': 110})

In [118]:
list(ex9[19].head.lefts)

[18,000]

In [120]:
extract_emp_relations(ex9)

Dep_ of EMP_NOUN is: dobj
Num_toks are: [19,000]
Root is at 1 steps from employees.
Dep_ of EMP_NOUN is: attr
Root is at 1 steps from employees.


[(we, had, 19,000, (0, 'Other'), employees),
 (we, had, approximately 18,000, (full-time, 'Full-Time'), employees)]

### Testing on sentences

In [18]:
test_sents = ["As of September 30, 2016, we employed approximately 7,300 employees world-wide.", 
"As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.", 
"At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.", 
"At December 31, 2016, we had approximately 9,400 full-time employees.", 
"As of October 29, 2016, we employed approximately 10,000 individuals worldwide.", 
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015.", 
"As of December 31, 2016, we had 1,469 total employees.", 
"ADP employed approximately 57,000 persons as of June 30, 2016.", 
"At December 31, 2016, we employed approximately 26,400 employees.", 
"The Company and its subsidiaries employed 1,562 persons at December 31, 2016, 114 of whom are covered by a collective bargaining agreement with District 10 of the International Association of Machinists.", 
"As of December 31, 2016, the Company had 455 employees, an increase of 17 employees from the prior year end.", 
"As of December 31, 2016, we had approximately 17,500 employees worldwide.", 
"Based in Neenah, Wisconsin, at December 31, 2016, the Company employed approximately 17,500 individuals and had 59 manufacturing facilities.", 
"As of January 31, 2017, we employed 7,683 individuals.", 
"At December 31, 2016, Bio-Rad had approximately 8,250 employees.", 
"As of December 31, 2016, we had approximately 8,500 full-time employees and 600 contractors.", 
"Alcoa's total worldwide employment at the end of 2016 was approximately 14,000 employees in 15 countries.", 
"As of December 31, 2016, we employed approximately 2,100 people.", 
"At December 31, 2016, the Company had approximately 11,500 employees.",
"As of February 23, 2017, we employed approximately 41,000 full-time Team Members and approximately 33,000 part-time Team Members.", 
"As of December 31, 2016, we had 699 full-time employees and 202 temporary employees.", 
"As of September 30, 2016, we had approximately 19,000 employees, of which approximately 18,000 were full-time employees.", 
"As of December 31, 2016, we had 2,646 employees, 1,581 of whom were pilots.", 
"The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively."
             ]

Nominal subjects are almost always left of root token. 

In [261]:
for i,t in enumerate(test_sents):
    nsubs = [t for t in nlp(t) if t.dep_ == 'nsubj']
    if len(nsubs) > 1:
        print(str(i))
        for i2, tok in enumerate(nsubs):
            print("Tok "+str(i2 + 1) + " of " + str(len(nsubs)))
            print("Token root: " + str(find_root_tok(tok)[0]))
            print("Token side of root: " + find_tok_side_of_root(tok, find_root_tok(tok)[0]))
            print("Sentence is :" + str(nlp(t).text))
            print("Token, dep_, index :" + str(tok) + " " + str(tok.dep_) + " " + str(tok.i))
            print("POS, tag, lemma :" + str(tok.pos_) + " " + str(tok.tag_) + " " + str(tok.lemma_))

21
Tok 1 of 2
Token root: had
Token side of root: left
Sentence is :As of December 31, 2016, we had 2,646 employees, 1,581 of whom were pilots.
Token, dep_, index :we nsubj 7
POS, tag, lemma :PRON PRP -PRON-
Tok 2 of 2
Token root: had
Token side of root: right
Sentence is :As of December 31, 2016, we had 2,646 employees, 1,581 of whom were pilots.
Token, dep_, index :1,581 nsubj 12
POS, tag, lemma :NUM CD 1,581


In [253]:
for i,t in enumerate(test_sents):
    print(str(i))
    emp_nouns = [t for t in nlp(t) if t.ent_type_ == 'EMP_NOUN']
    for i2, tok in enumerate(emp_nouns):
        print("Tok "+str(i2 + 1) + " of " + str(len(emp_nouns)))
        print("Sentence is :" + str(nlp(t).text))
        print("Token, dep_, index :" + str(tok) + " " + str(tok.dep_) + " " + str(tok.i))
        print("POS, tag, lemma :" + str(tok.pos_) + " " + str(tok.tag_) + " " + str(tok.lemma_))

0
Tok 1 of 1
Sentence is :As of September 30, 2016, we employed approximately 7,300 employees world-wide.
Token, dep_, index :employees dobj 11
POS, tag, lemma :NOUN NNS employee
1
Tok 1 of 1
Sentence is :As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.
Token, dep_, index :employees pobj 16
POS, tag, lemma :NOUN NNS employee
2
Tok 1 of 1
Sentence is :At December 31, 2016 and 2015, we had approximately 56,400 and 66,400 employees, respectively.
Token, dep_, index :employees dobj 14
POS, tag, lemma :NOUN NNS employee
3
Tok 1 of 1
Sentence is :At December 31, 2016, we had approximately 9,400 full-time employees.
Token, dep_, index :employees dobj 13
POS, tag, lemma :NOUN NNS employee
4
Tok 1 of 1
Sentence is :As of October 29, 2016, we employed approximately 10,000 individuals worldwide.
Token, dep_, index :individuals dobj 11
POS, tag, lemma :NOUN NNS individual
5
Tok 1 of 1
Sentence is :The number of full-time employees of the Company was approximately 3

In [69]:
extract_emp_relations(nlp(test_sents[5]), verbose=False)

[[(0,
   0,
   number,
   was,
   'approximately 31,800',
   'Full-Time',
   full-time,
   employees)]]

In [84]:
for i,t in enumerate(test_sents):
    print(str(i))
    print(extract_emp_relations(nlp(t), verbose=False))

0
[[(0, 0, we, employed, 7,300, 'Other', 0, employees)]]
1
[[(0, 0, subsidiaries, had, 17,634, 'Other', 0, employees)]]
2
[[(0, 0, we, had, '56,400', 'Other', 0, employees)]]
3
[[(0, 0, we, had, 9,400, 'Full-Time', full-time, employees)]]
4
[[(0, 0, we, employed, 10,000, 'Other', 0, individuals)]]
5
[[(0, 0, number, was, 'approximately 31,800', 'Full-Time', full-time, employees)]]
6
[[(0, 0, we, had, 1,469, 'Other', total, employees)]]
7
[[(0, 0, ADP, employed, 57,000, 'Other', 0, persons)]]
8
[[(0, 0, we, employed, 26,400, 'Other', 0, employees)]]
9
[[(0, 0, Company, employed, 1,562, 'Other', 0, persons)]]
10
[[(0, 0, Company, had, 455, 'Other', 0, employees), (0, 1, Company, had, 17, 'Other', 0, employees)]]
11
[[(0, 0, we, had, 17,500, 'Other', 0, employees)]]
12
[[(0, 0, Company, employed, 17,500, 'Other', 0, individuals)]]
13
[[(0, 0, we, employed, 7,683, 'Other', 0, individuals)]]
14
[[(0, 0, Bio-Rad, had, 8,250, 'Other', 0, employees)]]
15
[[(0, 0, we, had, 8,500, 'Full-Time', f

## Testing with paragraphs

acc_id: `0000034088-17-000017`  
```
"The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively. Regular employees are defined as active executive, management, professional, technical and wage employees who work full time or part time for the Corporation and are covered by the Corporation's benefit plans and programs. Regular employees do not include employees of the company‑operated retail sites (CORS). The number of CORS employees was 1.6 thousand, 2.1 thousand, and 8.4 thousand at years ended 2016, 2015 and 2014, respectively. The decrease in CORS employees reflects the multi‑year transition of the company‑operated retail network to a more capital‑efficient Branded Wholesaler model."
```

In [56]:
test_paragraphs = ["The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively. Regular employees are defined as active executive, management, professional, technical and wage employees who work full time or part time for the Corporation and are covered by the Corporation's benefit plans and programs. Regular employees do not include employees of the company‑operated retail sites (CORS). The number of CORS employees was 1.6 thousand, 2.1 thousand, and 8.4 thousand at years ended 2016, 2015 and 2014, respectively. The decrease in CORS employees reflects the multi‑year transition of the company‑operated retail network to a more capital‑efficient Branded Wholesaler model."]

In [85]:
extract_emp_relations(nlp(test_paragraphs[0]), verbose=False)

[[(0, 0, number, was, '71.1 thousand', 'Other', regular, employees)],
 [(3, 0, number, was, '1.6 thousand', 'Other', CORS, employees)]]

In [87]:
rels = extract_emp_relations(nlp(test_paragraphs[0]), verbose=False)

In [88]:
for rel in rels:
    sent_num, word_num, s, v, quantity, quantity_type, type_token ,word = rel[0]
    print(sent_num, word_num, s, v, quantity, quantity_type, type_token , word)

0 0 number was 71.1 thousand Other regular employees
3 0 number was 1.6 thousand Other CORS employees


In [75]:
print_df(make_tok_df(nlp(test_paragraphs[0])))

,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,,The,the,det,number,nsubj,DET,DT,determiner,determiner
1,,number,number,nsubj,was,ROOT,NOUN,NN,nominal subject,"noun, singular or mass"
2,,of,of,prep,number,nsubj,ADP,IN,prepositional modifier,"conjunction, subordinating or preposition"
3,,regular,regular,amod,employees,pobj,ADJ,JJ,adjectival modifier,adjective
4,EMP_NOUN,employees,employee,pobj,of,prep,NOUN,NNS,object of preposition,"noun, plural"
5,,was,be,ROOT,was,ROOT,VERB,VBD,None,"verb, past tense"
6,CARDINAL,71.1,71.1,compound,thousand,attr,NUM,CD,None,cardinal number
7,CARDINAL,thousand,thousand,attr,was,ROOT,NUM,CD,attribute,cardinal number
8,,",",",",punct,thousand,attr,PUNCT,",",punctuation,"punctuation mark, comma"
9,CARDINAL,73.5,73.5,compound,thousand,appos,NUM,CD,None,cardinal number


In [89]:
print_doc_info(nlp(test_paragraphs[0]))

doc is: 
The number of regular employees was 71.1 thousand, 73.5 thousand, and 75.3 thousand at years ended 2016, 2015 and 2014, respectively. Regular employees are defined as active executive, management, professional, technical and wage employees who work full time or part time for the Corporation and are covered by the Corporation's benefit plans and programs. Regular employees do not include employees of the company‑operated retail sites (CORS). The number of CORS employees was 1.6 thousand, 2.1 thousand, and 8.4 thousand at years ended 2016, 2015 and 2014, respectively. The decrease in CORS employees reflects the multi‑year transition of the company‑operated retail network to a more capital‑efficient Branded Wholesaler model.
--------------------------------------------------
Entities are: 


,tok_i,entity,ent_label,root,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,4,employees,EMP_NOUN,employees,pobj,object of preposition,of,prep,ADP
1,6,71.1 thousand,CARDINAL,thousand,attr,attribute,was,ROOT,VERB
2,9,73.5 thousand,CARDINAL,thousand,appos,appositional modifier,thousand,attr,NUM
3,13,75.3 thousand,CARDINAL,thousand,conj,conjunct,thousand,attr,NUM
4,16,years ended 2016,DATE,ended,advcl,adverbial clause modifier,was,ROOT,VERB
5,20,2015,DATE,2015,conj,conjunct,2016,npadvmod,NUM
6,22,2014,DATE,2014,conj,conjunct,2015,conj,NUM
7,27,employees,EMP_NOUN,employees,nsubjpass,nominal subject (passive),defined,ROOT,VERB
8,41,employees,EMP_NOUN,employees,appos,appositional modifier,executive,pobj,NOUN
9,44,full time,FULL_TIME,full time,npadvmod,noun phrase as adverbial modifier,work,relcl,VERB


--------------------------------------------------
Noun chunks are: 


,tok_i,noun_chunk,root,root_ent,root_dep,dep_def,root_head,root_head_dep,root_head_pos
0,0,The number,number,,nsubj,nominal subject,was,ROOT,VERB
1,3,regular employees,employees,EMP_NOUN,pobj,object of preposition,of,prep,ADP
2,16,years,years,DATE,pobj,object of preposition,at,prep,ADP
3,26,Regular employees,employees,EMP_NOUN,nsubjpass,nominal subject (passive),defined,ROOT,VERB
4,31,active executive,executive,,pobj,object of preposition,as,prep,ADP
5,34,management,management,,conj,conjunct,executive,pobj,NOUN
6,40,wage,wage,,conj,conjunct,technical,conj,ADJ
7,41,employees,employees,EMP_NOUN,appos,appositional modifier,executive,pobj,NOUN
8,42,who,who,,nsubj,nominal subject,work,relcl,VERB
9,48,the Corporation,Corporation,ORG,pobj,object of preposition,for,dative,ADP


--------------------------------------------------
Cardinal entities are: 


,tok_ent,toks,lemma,dep,head,h_dep,pos,tag,dep_def,tag_def
0,CARDINAL,71.1,71.1,compound,thousand,attr,NUM,CD,None,cardinal number
1,CARDINAL,thousand,thousand,attr,was,ROOT,NUM,CD,attribute,cardinal number
2,CARDINAL,73.5,73.5,compound,thousand,appos,NUM,CD,None,cardinal number
3,CARDINAL,thousand,thousand,appos,thousand,attr,NUM,CD,appositional modifier,cardinal number
4,CARDINAL,75.3,75.3,compound,thousand,conj,NUM,CD,None,cardinal number
5,CARDINAL,thousand,thousand,conj,thousand,attr,NUM,CD,conjunct,cardinal number
6,CARDINAL,1.6,1.6,compound,thousand,attr,NUM,CD,None,cardinal number
7,CARDINAL,thousand,thousand,attr,was,ROOT,NUM,CD,attribute,cardinal number
8,CARDINAL,2.1,2.1,compound,thousand,appos,NUM,CD,None,cardinal number
9,CARDINAL,thousand,thousand,appos,thousand,attr,NUM,CD,appositional modifier,cardinal number


In [76]:
list(nlp(test_paragraphs[0])[81].children)

[CORS]

In [13]:
train_df[train_df.acc_id == '0000034088-17-000017']

,acc_id,para_text,len,emp_header,first_emp_head_block,para_text_orig,para_tag,split,label


In [15]:
train_para_list = train_df.para_text.tolist()

In [15]:
test_para = nlp(train_para_list[0])

In [20]:
train_para_list[0]

'As of September 30, 2016, we employed approximately 7,300 employees world-wide. Approximately 860 of our employees in Mexico, 450 employees in Singapore, and 200 employees in Japan are covered by collective bargaining and other union agreements.'

In [90]:
extract_emp_relations(nlp(train_para_list[0]), verbose=False)

[[(0, 0, we, employed, 7,300, 'Other', 0, employees)]]

In [93]:
test_list = ["We are a small company with approximately 61 employees.",
"Total workforce level at December 31, 2016 was approximately 150,500",           
"Currently, the Company and its subsidiaries have an aggregate of 35 employees.",
"we employ only 31 employees", 
"We currently have 21 employees",
"We currently employ 26 full-time employees",
"Including our full and part-time personnel, we estimate that we have \
the equivalent of 12 full time employees.",
"As a REIT, we employ only 31 employees and have a cost-effective \
management structure."
            ]

In [94]:
for i,t in enumerate(test_list):
    print(str(i))
    print(extract_emp_relations(nlp(t), verbose=True))

0
Dep_ of EMP_NOUN is: pobj
Num_toks are: [61]
Root is at 1 steps from employees.
(We, are, 61, 'Other', 0, employees)
[[(0, 0, We, are, 61, 'Other', 0, employees)]]
1
[[]]
2
Dep_ of EMP_NOUN is: pobj
Num_toks are: [35]
Root is at 1 steps from employees.
(Company, have, 35, 'Other', 0, employees)
[[(0, 0, Company, have, 35, 'Other', 0, employees)]]
3
Dep_ of EMP_NOUN is: dobj
Num_toks are: [31]
Root is at 1 steps from employees.
(we, employ, 31, 'Other', 0, employees)
[[(0, 0, we, employ, 31, 'Other', 0, employees)]]
4
Dep_ of EMP_NOUN is: dobj
Num_toks are: [21]
Root is at 1 steps from employees.
(We, have, 21, 'Other', 0, employees)
[[(0, 0, We, have, 21, 'Other', 0, employees)]]
5
Dep_ of EMP_NOUN is: dobj
Num_toks are: [26]
Root is at 1 steps from employees.
(We, employ, 26, 'Full-Time', full-time, employees)
[[(0, 0, We, employ, 26, 'Full-Time', full-time, employees)]]
6
Root token lemma not one of ['be', 'employ', 'have']. 
Root token, lemma are : estimate estimate
[Including, ou

## Matcher testing

In [183]:
emp_ner_regex = re.compile(r"employees|individuals|people|persons|team members", re.I)

emp_flag = lambda text: bool(emp_ner_regex.match(text))

IS_EMP = nlp.vocab.add_flag(emp_flag)

matcher = Matcher(nlp.vocab)

num_emp_pat = [{'ENT_TYPE' : 'CARDINAL', 'TAG' : 'CD'}, {IS_EMP : True}]

matcher.add('NUM_EMP', None, num_emp_pat)
matcher.add('EMP_NOUN', None, [{IS_EMP: True}])

In [184]:
matches1 = matcher(doc1)

In [186]:
for match_id, start, end in matches1:
# create a new Span for each match and use the match_id as the label
    span = Span(doc1, start, end, label=match_id)
    doc1.ents = list(doc1.ents) + [span]  # add span to doc.ents

In [164]:
def add_match_ents(doc, matches):
    """Add matches from Matcher instance to a document's entities."""
    for match_id, start, end in matches:
    # create a new Span for each match and use the match_id as the label
        span = Span(doc, start, end, label=match_id)
        doc.ents = list(doc.ents) + [span]  # add span to doc.ents

In [166]:
matcher = Matcher(nlp.vocab)
matcher.add('NumEmp', None, num_emp_pat)

In [128]:
for match in re.finditer(emp_ner_regex, nlp(test_sents[0]).text):
    start, end = match.span()         # get matched indices
    span = doc.char_span(start, end)  # create Span from indices
    print(span.text)

employees


In [189]:
for match_id, start, end in matches1:
    span = doc1[start:end]
    print(span)
    print(span.sent)

17,634 employees
As of December 31, 2016, the subsidiaries of AEP had a total of 17,634 employees.
